In [1]:
import re
import unicodedata
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time
import json
import pandas as pd

In [2]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

dictio_scrap = {"nombre_marca": []}

driver.get(url)

nombre_marca = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-brand")))

for marca in nombre_marca:
    dictio_scrap["nombre_marca"].append(marca.text)

print(dictio_scrap)

driver.quit()

{'nombre_marca': ['TARTE', 'VALENTINO MAKE UP', 'FENTY BEAUTY', 'FENTY BEAUTY', 'CHARLOTTE TILBURY', 'CHARLOTTE TILBURY', 'NARS', 'NUDESTIX', 'TARTE', 'TARTE', 'TARTE', 'FENTY BEAUTY', 'TARTE', 'TARTE', 'NUDESTIX', 'NUDESTIX', 'TARTE', 'CHARLOTTE TILBURY', 'TOO FACED', 'BEAUTYBLENDER', 'MAKEUP BY MARIO', 'GLOW RECIPE', 'CHARLOTTE TILBURY', 'CHARLOTTE TILBURY']}


In [34]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

dictio_scrap = {"marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": []}

driver.get(url)

marcas = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-brand")))

nombres = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "summarize-description.title-line.title-line-bold")))

descripciones = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".summarize-description.title-line:not(.title-line-bold)")))

precios = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales-standard")))

cantidad_valoraciones = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "amount-of-reviews")))

for marca in marcas:
    dictio_scrap["marca"].append(marca.text)

for nombre in nombres:
    dictio_scrap["nombre"].append(nombre.text)

for descripcion in descripciones:
    dictio_scrap["descripcion"].append(descripcion.text)

for precio in precios:
    dictio_scrap["precio"].append(precio.text.replace(" €", ""))

for valoracion in cantidad_valoraciones:
    dictio_scrap["numero_valoraciones"].append(valoracion.text)

print(dictio_scrap)

driver.quit()

{'marca': ['FENTY BEAUTY', 'MAKEUP BY MARIO', 'FENTY BEAUTY', 'GLOW RECIPE', 'CHARLOTTE TILBURY', 'CHARLOTTE TILBURY', 'CHARLOTTE TILBURY', 'CHARLOTTE TILBURY', 'NARS', 'NUDESTIX', 'YVES SAINT LAURENT', 'SEPHORA COLLECTION', 'HAUS LABS', 'FENTY BEAUTY', 'GUERLAIN', 'TARTE', 'GIVENCHY', 'HUDA BEAUTY', 'HUDA BEAUTY', 'RARE BEAUTY', 'ERBORIAN', 'ERBORIAN', 'SEPHORA COLLECTION', 'MASQMAI'], 'nombre': ['Gloss Bomb Oil', 'SurrealSkin™ Soft Blur Setting Powder', 'Hella Thicc Extra Primer', 'Dewy Babies', 'Hot Lips 2.0', "Charlotte's Beauty Soulmates", 'Pillow Talk Lip & Cheek Set', 'Pillow Talk Love Efecto Lipstick', 'Sweet Kiss Quad Eyeshadow', 'Nudies All Over Face Color Bronze + Glow', 'Loveshine', 'Bálsamo Desmaquillante Piel Limpia', 'Bio-Radiant™ Glassy Highlighter Balm', 'Cherry Mouth', 'Terracotta Light', 'Maracuja Juicy Lip Duo', 'Rose Perfecto Shine Serum Lipstick', 'Easy Bake Setting Spray', 'Mini Puffs Powder Puff Trío', 'Soft Pinch Matte Bouncy Blush', 'Kit CC Crème x Super BB Co

In [38]:
print(len(dictio_scrap["numero_valoraciones"]))
print(len(dictio_scrap["precio"]))
print(len(dictio_scrap["marca"]))
print(dictio_scrap["numero_valoraciones"])

12
22
24
['2', '471', '16', '1', '472', '110', '39', '1', '270', '428', '1', '8']


In [39]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

dictio_scrap = {"marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": []}

driver.get(url)

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "product-info")))

info_productos = driver.find_elements(By.CLASS_NAME, "product-info")

for producto in info_productos:
    
    marca = producto.find_element(By.CLASS_NAME, "product-brand")
    dictio_scrap["marca"].append(marca.text)

    nombre = producto.find_element(By.CLASS_NAME, "summarize-description.title-line.title-line-bold")
    dictio_scrap["nombre"].append(nombre.text)

    descripcion = producto.find_element(By.CSS_SELECTOR, ".summarize-description.title-line:not(.title-line-bold)")
    dictio_scrap["descripcion"].append(descripcion.text)

    try:
        precio = producto.find_element(By.CLASS_NAME, "price-sales-standard")
        dictio_scrap["precio"].append(precio.text.replace(" €", ""))
    except NoSuchElementException:
        precio = producto.find_element(By.CLASS_NAME, "product-min-price")
        dictio_scrap["precio"].append(precio.text.replace(" €", ""))

    try: 
        valoracion = producto.find_element(By.CLASS_NAME, "amount-of-reviews")
        dictio_scrap["numero_valoraciones"].append(valoracion.text)
    except NoSuchElementException:
        dictio_scrap["numero_valoraciones"].append(0)

print(dictio_scrap)
    
driver.quit()

{'marca': ['FENTY BEAUTY', 'MAKEUP BY MARIO', 'FENTY BEAUTY', 'GLOW RECIPE', 'CHARLOTTE TILBURY', 'CHARLOTTE TILBURY', 'CHARLOTTE TILBURY', 'CHARLOTTE TILBURY', 'NARS', 'NUDESTIX', 'YVES SAINT LAURENT', 'SEPHORA COLLECTION', 'HAUS LABS', 'FENTY BEAUTY', 'GUERLAIN', 'TARTE', 'GIVENCHY', 'HUDA BEAUTY', 'HUDA BEAUTY', 'RARE BEAUTY', 'ERBORIAN', 'ERBORIAN', 'SEPHORA COLLECTION', 'MASQMAI'], 'nombre': ['Gloss Bomb Oil', 'SurrealSkin™ Soft Blur Setting Powder', 'Hella Thicc Extra Primer', 'Dewy Babies', 'Hot Lips 2.0', "Charlotte's Beauty Soulmates", 'Pillow Talk Lip & Cheek Set', 'Pillow Talk Love Efecto Lipstick', 'Sweet Kiss Quad Eyeshadow', 'Nudies All Over Face Color Bronze + Glow', 'Loveshine', 'Bálsamo Desmaquillante Piel Limpia', 'Bio-Radiant™ Glassy Highlighter Balm', 'Cherry Mouth', 'Terracotta Light', 'Maracuja Juicy Lip Duo', 'Rose Perfecto Shine Serum Lipstick', 'Easy Bake Setting Spray', 'Mini Puffs Powder Puff Trío', 'Soft Pinch Matte Bouncy Blush', 'Kit CC Crème x Super BB Co

In [40]:
print(len(dictio_scrap["marca"]))
print(len(dictio_scrap["nombre"]))
print(len(dictio_scrap["descripcion"]))
print(len(dictio_scrap["precio"]))
print(len(dictio_scrap["numero_valoraciones"]))

24
24
24
24
24


In [4]:
url = "https://www.sephora.es/p/charlottes-beauty-soulmates---paleta-rostro-P1000207296.html"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

valoracion = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).text

print(f"La valoración es: {valoracion}")

driver.quit()

La valoración es: 


In [2]:
url = "https://www.sephora.es/p/charlottes-beauty-soulmates---paleta-rostro-P1000207296.html"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

contenido = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')

print(contenido)

valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
    
print(f"La valoración es: {valoracion}")
    
driver.quit()


                
                    5.0/5
                
                <br>
                <span class="bv-number-review">
                    26 opiniones
                </span>
                <br>
                <div class="bv-stars-summary">
                    
                        
                            <img class="product-rating-icon" src="/on/demandware.static/Sites-Sephora_ES-Site/-/default/dw8b705fa9/images/svg-icons/rating-star-full-icon.svg" alt="Rating star">
                        
                    
                        
                            <img class="product-rating-icon" src="/on/demandware.static/Sites-Sephora_ES-Site/-/default/dw8b705fa9/images/svg-icons/rating-star-full-icon.svg" alt="Rating star">
                        
                    
                        
                            <img class="product-rating-icon" src="/on/demandware.static/Sites-Sephora_ES-Site/-/default/dw8b705fa9/images/svg-icons/rating-star-full-icon

In [3]:
url = "https://www.sephora.es/p/charlottes-beauty-soulmates---paleta-rostro-P1000207296.html"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

tonos = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]

print(f"Tiene {tonos} tonos disponibles")
    
driver.quit()

Tiene 2 tonos disponibles


In [11]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

driver.quit()

for url in productos_urls:
    print(url)

https://www.sephora.es/p/gloss-bomb-oil---aceite-gloss-para-los-labios-P1000207159.html
https://www.sephora.es/p/surrealskin%e2%84%a2-soft-blur-setting-powder---polvos-libres-P10061412.html
https://www.sephora.es/p/hella-thicc-extra-primer---base-voluminizadora-para-m%c3%a1scaras-de-pestanas-760348.html
https://www.sephora.es/p/dewy-babies---conjunto-best-seller-758028.html
https://www.sephora.es/p/hot-lips-2.0---barra-de-labios-P3766008.html
https://www.sephora.es/p/charlottes-beauty-soulmates---paleta-rostro-P1000207296.html
https://www.sephora.es/p/pillow-talk-lip-cheek-set---estuche-labios-y-mejillas-P1000207383.html
https://www.sephora.es/p/charlottes-beauty-soulmates-face-palette---paleta-rostro-P1000207297.html
https://www.sephora.es/p/sweet-kiss-quad-eyeshadow---paleta-de-4-sombras-de-ojos-757302.html
https://www.sephora.es/p/nudies-all-over-face-color-bronze-glow---stick-bronceador-luminosidad-P3006022.html
https://www.sephora.es/p/loveshine---gloss-P1000206779.html
https://ww

In [ ]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

driver.quit()

dictio_scrap = {"valoracion": [], "num_tonos": [], "num_tamaños": []}

for product_url in productos_urls:
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(product_url)

    try:
        contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        dictio_scrap["valoracion"].append(valoracion)
    except TimeoutException:
        dictio_scrap["valoracion"].append(0)
    except Exception as e:
        dictio_scrap["valoracion"].append(0)

    try:
        tonos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        dictio_scrap["num_tonos"].append(tonos)
    except Exception as e:
        dictio_scrap["num_tonos"].append("No dispone de varios tonos")

    try:
        tamaños = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        dictio_scrap["num_tamaños"].append(tamaños)
    except Exception as e:
        dictio_scrap["num_tamaños"].append("No dispone de varios tamaños")

    driver.quit()

print(dictio_scrap)

{'valoracion': [0, '4.5', 0, 0, '4.4', '5.0', 0, 0, '3.0', '4.4', '4.7', '4.6', '5.0', 0, 0, 0, '4.7', '4.8', 0, '5.0', 0, 0, '4.6', 0], 'num_tonos': ['Selecciona un tono (7)', 'Selecciona un tono (7)', 'No dispone de varios tonos', 'No dispone de varios tonos', 'Selecciona un tono (7)', 'Selecciona un tono (2)', 'Selecciona un tono (2)', 'Selecciona un tono (4)', 'No dispone de varios tonos', 'Selecciona un tono (4)', 'Selecciona un tono (8)', 'No dispone de varios tonos', 'Selecciona un tono (6)', 'No dispone de varios tonos', 'No dispone de varios tonos', 'Selecciona un tono (3)', 'Selecciona un tono (7)', 'No dispone de varios tonos', 'No dispone de varios tonos', 'Selecciona un tono (5)', 'No dispone de varios tonos', 'No dispone de varios tonos', 'Selecciona un tono (4)', 'Selecciona un tono (4)'], 'num_tamaños': ['No dispone de varios tamaños', 'No dispone de varios tamaños', 'No dispone de varios tamaños', 'No dispone de varios tamaños', 'No dispone de varios tamaños', 'No disp

In [ ]:
df = pd.DataFrame(dictio_scrap)
df.head()

,valoracion,num_tonos,num_tamaños
0,0,Selecciona un tono (7),No dispone de varios tamaños
1,4.5,Selecciona un tono (7),No dispone de varios tamaños
2,0,No dispone de varios tonos,No dispone de varios tamaños
3,0,No dispone de varios tonos,No dispone de varios tamaños
4,4.4,Selecciona un tono (7),No dispone de varios tamaños
5,5.0,Selecciona un tono (2),No dispone de varios tamaños
6,0,Selecciona un tono (2),No dispone de varios tamaños
7,0,Selecciona un tono (4),No dispone de varios tamaños
8,3.0,No dispone de varios tonos,No dispone de varios tamaños
9,4.4,Selecciona un tono (4),No dispone de varios tamaños


In [13]:
url = "https://www.sephora.es/p/charlottes-beauty-soulmates---paleta-rostro-P1000207296.html"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text

nombre, descripcion = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text.split(" - ")

precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")

numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]

print(f"Marca: {marca}")
print(f"Nombre: {nombre}")
print(f"Descripción: {descripcion}")
print(f"Precio: {precio}")
print(f"Número de valoraciones: {numero_valoraciones}")

driver.quit()

Marca: Charlotte Tilbury
Nombre: Charlotte's Beauty Soulmates
Descripción: Paleta rostro
Precio: 53,00
Número de valoraciones: 17


In [14]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

driver.quit()

dictio_scrap = {"marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": [], "valoracion": [], "num_tonos": [], "num_tamaños": []}

for product_url in productos_urls:
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(product_url)

    marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text
    dictio_scrap["marca"].append(marca)

    nombre, descripcion = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text.split(" - ")
    dictio_scrap["nombre"].append(nombre)
    dictio_scrap["descripcion"].append(descripcion)

    precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")
    dictio_scrap["precio"].append(precio)

    numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]
    dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

    try:
        contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        dictio_scrap["valoracion"].append(valoracion)
    except TimeoutException:
        dictio_scrap["valoracion"].append(0)
    except Exception as e:
        dictio_scrap["valoracion"].append(0)

    try:
        tonos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        dictio_scrap["num_tonos"].append(tonos)
    except Exception as e:
        dictio_scrap["num_tonos"].append("No dispone de varios tonos")

    try:
        tamaños = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        dictio_scrap["num_tamaños"].append(tamaños)
    except Exception as e:
        dictio_scrap["num_tamaños"].append("No dispone de varios tamaños")

    driver.quit()

print(dictio_scrap)

{'marca': ['Fenty Beauty', 'Valentino Make Up', 'Tarte', 'Makeup By Mario', 'Fenty Beauty', 'Glow Recipe', 'Charlotte Tilbury', 'Charlotte Tilbury', 'Charlotte Tilbury', 'Charlotte Tilbury', 'Nars', 'Nudestix', 'Yves Saint Laurent', 'Sephora Collection', 'Haus Labs', 'Guerlain', 'Fenty Beauty', 'Tarte', 'Givenchy', 'Huda Beauty', 'Huda Beauty', 'Rare Beauty', 'Erborian', 'Erborian'], 'nombre': ['Gloss Bomb Oil', 'Puffer Gloss', 'maracuja juicy melt mask', 'SurrealSkin™ Soft Blur Setting Powder', 'Hella Thicc Extra Primer', 'Dewy Babies', 'Hot Lips 2.0', 'Pillow Talk Lip & Cheek Set', "Charlotte's Beauty Soulmates", 'Pillow Talk Love Efecto Lipstick', 'Sweet Kiss Quad Eyeshadow', 'Nudies All Over Face Color Bronze + Glow', 'Loveshine', 'Bálsamo Desmaquillante Piel Limpia', 'Bio-Radiant™ Glassy Highlighter Balm', 'Terracotta Light', 'Cherry Mouth', 'Maracuja Juicy Lip Duo', 'Rose Perfecto Shine Serum Lipstick', 'Easy Bake Setting Spray', 'Mini Puffs Powder Puff Trío', 'Soft Pinch Matte B

In [15]:
df = pd.DataFrame(dictio_scrap)
df.head()

,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_tonos,num_tamaños
0,Fenty Beauty,Gloss Bomb Oil,Aceite-gloss para los labios,"28,00",0,0,7,No dispone de varios tamaños
1,Valentino Make Up,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...","39,99",0,0,5,No dispone de varios tamaños
2,Tarte,maracuja juicy melt mask,Mascarilla labial,"27,00",0,0,No dispone de varios tonos,3
3,Makeup By Mario,SurrealSkin™ Soft Blur Setting Powder,Polvos Libres,"43,00",2,4.5,7,No dispone de varios tamaños
4,Fenty Beauty,Hella Thicc Extra Primer,Base voluminizadora para máscaras de pestañas,"21,00",0,0,No dispone de varios tonos,No dispone de varios tamaños


In [4]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(100):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))

    if new_height == last_height and i > 5:
        break
    last_height = new_height

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

print(len(productos_urls))

driver.quit()

1915


In [3]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(100):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))

    if new_height == last_height and i > 5:
        break
    last_height = new_height

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

productos_urls = productos_urls[:100]

print(len(productos_urls))

driver.quit()

100


In [5]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(100):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))

    if new_height == last_height and i > 5:
        break
    last_height = new_height

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

productos_urls = productos_urls[:100]

print(len(productos_urls))

driver.quit()

dictio_scrap = {"marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": [], "valoracion": [], "num_tonos": [], "num_tamaños": []}

driver = webdriver.Chrome(service=service, options=options)

for product_url in productos_urls:
    driver.get(product_url)

    marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text
    dictio_scrap["marca"].append(marca)

    titulo = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text

    if " - " in titulo:
        nombre, descripcion = titulo.split(" - ")
    else:
        nombre = titulo
        descripcion = "Sin descripción"

    dictio_scrap["nombre"].append(nombre)
    dictio_scrap["descripcion"].append(descripcion)

    precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")
    dictio_scrap["precio"].append(precio)

    numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]
    dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

    try:
        contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        dictio_scrap["valoracion"].append(valoracion)
    except TimeoutException:
        dictio_scrap["valoracion"].append(0)
    except Exception as e:
        dictio_scrap["valoracion"].append(0)

    try:
        tonos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        dictio_scrap["num_tonos"].append(tonos)
    except Exception as e:
        dictio_scrap["num_tonos"].append("No dispone de varios tonos")

    try:
        tamaños = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        dictio_scrap["num_tamaños"].append(tamaños)
    except Exception as e:
        dictio_scrap["num_tamaños"].append("No dispone de varios tamaños")

driver.quit()

print(len(dictio_scrap["marca"]))

100
100


In [6]:
df = pd.DataFrame(dictio_scrap)
df

,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_tonos,num_tamaños
0,Tarte,maracuja juicy melt mask,Mascarilla labial,"27,00",0,0,No dispone de varios tonos,3
1,Valentino Make Up,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...","39,99",0,0,5,No dispone de varios tamaños
2,Fenty Beauty,Hella Thicc Extra Primer,Base voluminizadora para máscaras de pestañas,"21,00",0,0,No dispone de varios tonos,No dispone de varios tamaños
3,Fenty Beauty,Gloss Bomb Oil,Aceite-gloss para los labios,"28,00",0,0,7,No dispone de varios tamaños
4,Charlotte Tilbury,Pillow Talk Lip & Cheek Set,Estuche labios y mejillas,"55,00",0,0,2,No dispone de varios tamaños
...,...,...,...,...,...,...,...,...
95,Charlotte Tilbury,Beautiful Skin Island Glow Easy Tanning Drops,Gotas autobronceadoras,"47,00",147,4.6,No dispone de varios tonos,2
96,Dermalogica,Special Cleansing Gel,Recarga Gel Limpiador,"65,00",0,0,No dispone de varios tonos,No dispone de varios tamaños
97,Huda Beauty,Easy Blur Natural Airbrush Foundation,Base de maquillaje,"39,00",268,4.6,30,No dispone de varios tamaños
98,Dior,Dior Forever Glow Luminizer,Iluminador infusionado con ácido hialurónico,"60,00",0,0,4,No dispone de varios tamaños


In [11]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

categoria_elementos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'refinement-link category-search-link')]")))

categorias = {}

for elem in categoria_elementos:
    href = elem.get_attribute("href")
    categoria_texto = elem.text
    categorias[categoria_texto] = href

categorias_filtradas = dict(list(categorias.items())[2:11])

print(categorias_filtradas)

driver.quit()


{'ROSTRO': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro-c342/', 'LABIOS': 'https://www.sephora.es/todos-los-productos/maquillaje/labios-c344/', 'OJOS': 'https://www.sephora.es/todos-los-productos/maquillaje/ojos-c343/', 'PINCELES & ACCESORIOS': 'https://www.sephora.es/todos-los-productos/maquillaje/pinceles-accesorios-c7631/', 'PALETAS MAQUILLAJE': 'https://www.sephora.es/todos-los-productos/maquillaje/paletas-maquillaje-c347/', 'CEJAS': 'https://www.sephora.es/todos-los-productos/maquillaje/cejas-c367/', 'UÑAS': 'https://www.sephora.es/todos-los-productos/maquillaje/unas-c345/', 'DESMAQUILLANTE': 'https://www.sephora.es/todos-los-productos/maquillaje/desmaquillante-c7632/', 'ESTUCHES MAQUILLAJE': 'https://www.sephora.es/todos-los-productos/maquillaje/estuches-maquillaje-c313702/'}


In [13]:
categorias_filtradas.keys()

dict_keys(['ROSTRO', 'LABIOS', 'OJOS', 'PINCELES & ACCESORIOS', 'PALETAS MAQUILLAJE', 'CEJAS', 'UÑAS', 'DESMAQUILLANTE', 'ESTUCHES MAQUILLAJE'])

In [16]:
url = "https://www.sephora.es/todos-los-productos/maquillaje/rostro-c342/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

subcategoria_elementos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'refinement-link category-search-link')]")))

subcategorias = {}

for elem in subcategoria_elementos:
    href = elem.get_attribute("href")
    subcategoria_texto = elem.text
    subcategorias[subcategoria_texto] = href

subcategorias_mayus = {k: v for k, v in subcategorias.items() if k.isupper()}

print(subcategorias_mayus)

driver.quit()

{'COLORETE': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/colorete-c356/', 'BASE DE MAQUILLAJE': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/base-de-maquillaje-c353/', 'ANTIOJERAS & CORRECTOR': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/antiojeras-corrector-c352/', 'ILUMINADOR': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/iluminador-c359/', 'PRIMER & SPRAY FIJADOR': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/primer-spray-fijador-c351/', 'POLVOS BRONCEADORES': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/polvos-bronceadores-c119401/', 'CONTOURING': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/contouring-c195701/', 'CREMAS BB & CC': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/cremas-bb-cc-c93801/', 'POLVOS MATIFICANTES': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/polvos-matificantes-c354/', 'POLVOS SUELTOS': 'https://www.sephora

In [18]:
subcategorias_mayus.keys()

dict_keys(['COLORETE', 'BASE DE MAQUILLAJE', 'ANTIOJERAS & CORRECTOR', 'ILUMINADOR', 'PRIMER & SPRAY FIJADOR', 'POLVOS BRONCEADORES', 'CONTOURING', 'CREMAS BB & CC', 'POLVOS MATIFICANTES', 'POLVOS SUELTOS', 'PALETA ROSTRO', 'CREMA DE COLOR'])

In [ ]:
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"
driver.get(url)

categoria_elementos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'refinement-link category-search-link')]")))

categorias = {}

for elem in categoria_elementos:
    href = elem.get_attribute("href")  
    if not href:  
        href = elem.get_attribute("data-href")
    
    categoria_texto = elem.text
    if href:  
        categorias[categoria_texto] = href

categorias_filtradas = dict(list(categorias.items())[2:11])

subcategorias_por_categoria = {}

for nombre_categoria, url_categoria in categorias_filtradas.items():
    driver.get(url_categoria)
    subcategoria_elementos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'refinement-link category-search-link')]")))

    subcategorias = {}

    for elem in subcategoria_elementos:
        href = elem.get_attribute("href")  
        if not href: 
            href = elem.get_attribute("data-href")
        
        subcategoria_texto = elem.text
        
        if href:  
            subcategorias[subcategoria_texto] = href

    subcategorias_mayus = {k: v for k, v in subcategorias.items() if k.isupper()}

    subcategorias_por_categoria[nombre_categoria] = subcategorias_mayus

driver.quit()

print(subcategorias_por_categoria)

{'ROSTRO': {'COLORETE': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/colorete-c356/', 'BASE DE MAQUILLAJE': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/base-de-maquillaje-c353/', 'ANTIOJERAS & CORRECTOR': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/antiojeras-corrector-c352/', 'ILUMINADOR': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/iluminador-c359/', 'PRIMER & SPRAY FIJADOR': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/primer-spray-fijador-c351/', 'POLVOS BRONCEADORES': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/polvos-bronceadores-c119401/', 'CONTOURING': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/contouring-c195701/', 'CREMAS BB & CC': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/cremas-bb-cc-c93801/', 'POLVOS MATIFICANTES': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/polvos-matificantes-c354/', 'POLVOS SUELTOS': 'https://

In [7]:
subcategorias_por_categoria["ROSTRO"]

{'COLORETE': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/colorete-c356/',
 'BASE DE MAQUILLAJE': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/base-de-maquillaje-c353/',
 'ANTIOJERAS & CORRECTOR': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/antiojeras-corrector-c352/',
 'ILUMINADOR': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/iluminador-c359/',
 'PRIMER & SPRAY FIJADOR': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/primer-spray-fijador-c351/',
 'POLVOS BRONCEADORES': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/polvos-bronceadores-c119401/',
 'CONTOURING': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/contouring-c195701/',
 'CREMAS BB & CC': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/cremas-bb-cc-c93801/',
 'POLVOS MATIFICANTES': 'https://www.sephora.es/todos-los-productos/maquillaje/rostro/polvos-matificantes-c354/',
 'POLVOS SUELTOS': 'https://ww

In [3]:
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

try:
    url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"
    driver.get(url)

    categoria_elementos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'refinement-link category-search-link')]")))

    categorias = {}
    for elem in categoria_elementos:
        href = elem.get_attribute("href")
        if not href:
            href = elem.get_attribute("data-href")
        categoria_texto = elem.text
        if href:
            categorias[categoria_texto] = href

    categorias_filtradas = dict(list(categorias.items())[2:11])

    subcategorias_por_categoria = {}

    for nombre_categoria, url_categoria in categorias_filtradas.items():
        driver.get(url_categoria)

        subcategoria_elementos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'refinement-link category-search-link')]")))

        subcategorias = {}
        for elem in subcategoria_elementos:
            href = elem.get_attribute("href")
            if not href:
                href = elem.get_attribute("data-href")
            subcategoria_texto = elem.text
            if href:
                subcategorias[subcategoria_texto] = href

        subcategorias_mayus = {k: v for k, v in subcategorias.items() if k.isupper()}
        subcategorias_por_categoria[nombre_categoria] = subcategorias_mayus

    dictio_scrap = {"marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": [], "valoracion": [], "num_tonos": [], "num_tamaños": [], "categoria": [], "subcategoria": []}

    for subcategoria_nombre, subcategoria_url in subcategorias_por_categoria["ROSTRO"].items():
        print(f"Accediendo a la subcategoría: {subcategoria_nombre}, URL: {subcategoria_url}")
        driver.get(subcategoria_url)

        try:
            ver_mas_btn = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]"))
                )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
            time.sleep(2)
            driver.execute_script("arguments[0].click();", ver_mas_btn)
        except TimeoutException:
            print(f"No se encontró el botón 'ver más' en la subcategoría {subcategoria_nombre}")

        last_height = driver.execute_script("return document.body.scrollHeight")
        for i in range(100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(4)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height and i > 5:
                break
            last_height = new_height

        productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

        productos_urls = []
        for producto in productos:
            data_tcproduct = producto.get_attribute('data-tcproduct')
            product_data = json.loads(data_tcproduct)
            product_url = product_data.get('product_url_page')
            product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
            productos_urls.append(product_url)

        productos_urls = productos_urls[:5]
        print(f"Productos en la subcategoría '{subcategoria_nombre}':", len(productos_urls))

        for product_url in productos_urls:
            driver.get(product_url)

            marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text
            dictio_scrap["marca"].append(marca)

            titulo = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text

            if " - " in titulo:
                nombre, descripcion = titulo.split(" - ", 1)
            else:
                nombre = titulo
                descripcion = "Sin descripción"

            dictio_scrap["nombre"].append(nombre)
            dictio_scrap["descripcion"].append(descripcion)

            precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")
            dictio_scrap["precio"].append(precio)

            numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]
            dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

            try:
                contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
                valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
                dictio_scrap["valoracion"].append(valoracion)
            except:
                dictio_scrap["valoracion"].append(0)

            try:
                tonos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
                dictio_scrap["num_tonos"].append(tonos)
            except:
                dictio_scrap["num_tonos"].append("No dispone de varios tonos")

            try:
                tamaños = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
                dictio_scrap["num_tamaños"].append(tamaños)
            except:
                dictio_scrap["num_tamaños"].append("No dispone de varios tamaños")

            dictio_scrap["categoria"].append(nombre_categoria)
            dictio_scrap["subcategoria"].append(subcategoria_nombre)

    print("Cantidad total de productos scrapeados:", len(dictio_scrap["marca"]))

finally:
    driver.quit()

Accediendo a la subcategoría: COLORETE, URL: https://www.sephora.es/todos-los-productos/maquillaje/rostro/colorete-c356/
Productos en la subcategoría 'COLORETE': 5
Accediendo a la subcategoría: BASE DE MAQUILLAJE, URL: https://www.sephora.es/todos-los-productos/maquillaje/rostro/base-de-maquillaje-c353/
Productos en la subcategoría 'BASE DE MAQUILLAJE': 5
Accediendo a la subcategoría: ANTIOJERAS & CORRECTOR, URL: https://www.sephora.es/todos-los-productos/maquillaje/rostro/antiojeras-corrector-c352/
Productos en la subcategoría 'ANTIOJERAS & CORRECTOR': 5
Accediendo a la subcategoría: ILUMINADOR, URL: https://www.sephora.es/todos-los-productos/maquillaje/rostro/iluminador-c359/
Productos en la subcategoría 'ILUMINADOR': 5
Accediendo a la subcategoría: PRIMER & SPRAY FIJADOR, URL: https://www.sephora.es/todos-los-productos/maquillaje/rostro/primer-spray-fijador-c351/
Productos en la subcategoría 'PRIMER & SPRAY FIJADOR': 5
Accediendo a la subcategoría: POLVOS BRONCEADORES, URL: https://

In [8]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(100):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))

    if new_height == last_height and i > 5:
        break
    last_height = new_height

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

productos_urls = productos_urls[:100]

print(len(productos_urls))

driver.quit()

100


In [9]:
from selenium.common.exceptions import TimeoutException

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

ingredientes = {"ingredientes": []}

for url in productos_urls:
    driver.get(url)
    try:
        contenido = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ingredients-content"))
        ).get_attribute('innerHTML')
        ingredientes["ingredientes"].append(contenido)
    except TimeoutException:
        print(f"No se encontraron ingredientes en: {url}")
        continue

driver.quit()

print(ingredientes["ingredientes"])

['\n    \n        <p>Ingredients: Diisostearyl Malate, phytosteryl/isostearyl/cetyl/stearyl/behenyl dimer dilinoleate, hydrogenated polyisobutene, polyisobutene, octyldodecanol, cocos nucifera (coconut) oil, disteardimonium hectorite, butyrospermum parkii (shea) butter, silica silylate, flavor/aroma, mangifera indica (mango) seed butter, hydrogenated poly(c6-14 olefin), passiflora edulis seed oil, vitis vinifera (grape) seed oil, carica papaya (papaya) seed oil, citrullus lanatus (watermelon) seed oil, linum usitatissimum (linseed) seed oil, prunus armeniaca (apricot) kernel oil, prunus domestica seed oil, ribes nigrum (black currant) seed oil, rosa canina seed oil, rubus fruticosus (blackberry) seed oil, salvia hispanica seed oil, helianthus annuus (sunflower) seed oil, tocopheryl acetate, citrullus lanatus (watermelon) fruit extract, citrus limon (lemon) fruit extract, euterpe oleracea fruit extract, fragaria vesca (strawberry) fruit extract, lycium barbarum fruit extract, malpighia 

In [10]:
ingredientes["ingredientes"]

['\n    \n        <p>Ingredients: Diisostearyl Malate, phytosteryl/isostearyl/cetyl/stearyl/behenyl dimer dilinoleate, hydrogenated polyisobutene, polyisobutene, octyldodecanol, cocos nucifera (coconut) oil, disteardimonium hectorite, butyrospermum parkii (shea) butter, silica silylate, flavor/aroma, mangifera indica (mango) seed butter, hydrogenated poly(c6-14 olefin), passiflora edulis seed oil, vitis vinifera (grape) seed oil, carica papaya (papaya) seed oil, citrullus lanatus (watermelon) seed oil, linum usitatissimum (linseed) seed oil, prunus armeniaca (apricot) kernel oil, prunus domestica seed oil, ribes nigrum (black currant) seed oil, rosa canina seed oil, rubus fruticosus (blackberry) seed oil, salvia hispanica seed oil, helianthus annuus (sunflower) seed oil, tocopheryl acetate, citrullus lanatus (watermelon) fruit extract, citrus limon (lemon) fruit extract, euterpe oleracea fruit extract, fragaria vesca (strawberry) fruit extract, lycium barbarum fruit extract, malpighia 

In [ ]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(100):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))

    if new_height == last_height and i > 5:
        break
    last_height = new_height

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

productos_urls = productos_urls[:10]

print(len(productos_urls))

driver.quit()

dictio_scrap = {"marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": [], "valoracion": [], "num_tonos": [], "num_tamaños": [], "categoria": [], "subcategoria": []} 

driver = webdriver.Chrome(service=service, options=options)

for product_url in productos_urls:
    driver.get(product_url)

    marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text
    dictio_scrap["marca"].append(marca)

    titulo = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text

    if " - " in titulo:
        nombre, descripcion = titulo.split(" - ")
    else:
        nombre = titulo
        descripcion = "Sin descripción"

    dictio_scrap["nombre"].append(nombre)
    dictio_scrap["descripcion"].append(descripcion)

    precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")
    dictio_scrap["precio"].append(precio)

    numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]
    dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

    categoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[2].text
    dictio_scrap["categoria"].append(categoria)

    subcategoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[3].text
    dictio_scrap["subcategoria"].append(subcategoria)

    try:
        contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        dictio_scrap["valoracion"].append(valoracion)
    except TimeoutException:
        dictio_scrap["valoracion"].append(0)
    except Exception as e:
        dictio_scrap["valoracion"].append(0)

    try:
        tonos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        dictio_scrap["num_tonos"].append(tonos)
    except Exception as e:
        dictio_scrap["num_tonos"].append("No dispone de varios tonos")

    try:
        tamaños = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        dictio_scrap["num_tamaños"].append(tamaños)
    except Exception as e:
        dictio_scrap["num_tamaños"].append("No dispone de varios tamaños")

driver.quit()

In [7]:
df = pd.DataFrame(dictio_scrap)
df

,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_tonos,num_tamaños,categoria,subcategoria
0,Tarte,maracuja juicy melt mask,Mascarilla labial,"27,00",0,0,No dispone de varios tonos,3,Labios,Bálsamo labial
1,Valentino Make Up,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...","39,99",0,0,5,No dispone de varios tamaños,Labios,Gloss/ Brillo labial
2,Fenty Beauty,Hella Thicc Extra Primer,Base voluminizadora para máscaras de pestañas,"21,00",0,0,No dispone de varios tonos,No dispone de varios tamaños,Ojos,Máscara de pestañas
3,Fenty Beauty,Gloss Bomb Oil,Aceite-gloss para los labios,"28,00",0,0,7,No dispone de varios tamaños,Labios,Gloss/ Brillo labial
4,Charlotte Tilbury,Pillow Talk Lip & Cheek Set,Estuche labios y mejillas,"55,00",0,0,2,No dispone de varios tamaños,Estuches maquillaje,Estuches multi-usos
5,Charlotte Tilbury,Pillow Talk Love Efecto Lipstick,Lápiz de labios,"36,00",0,0,4,No dispone de varios tamaños,Labios,Barra de labios
6,Nars,Sweet Kiss Quad Eyeshadow,Paleta de 4 sombras de ojos,"52,00",1,3.0,No dispone de varios tonos,No dispone de varios tamaños,Ojos,Paleta de sombras de ojos
7,Nudestix,Nudies All Over Face Color Bronze + Glow,"Para ojos, mejillas y labios","33,99",475,4.4,4,No dispone de varios tamaños,Rostro,Colorete
8,Tarte,Life Lock™ hydrating setting spray,Spray fijador hidratante,"37,00",2,5.0,No dispone de varios tonos,2,Rostro,Primer & spray fijador
9,Tarte,Shape Tape™ Corrector,Corrector,"30,00",66,4.1,4,No dispone de varios tamaños,Rostro,Antiojeras & corrector


In [10]:
url = "https://www.sephora.es/p/maracuja-juicy-melt-mask---mascarilla-labial-P10061602.html"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "variation-title")))

formatos = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".variation-title span")))

for i, formato in enumerate(formatos):
    inner_html = formato.get_attribute("innerHTML").strip() 
    print(f"Elemento {i+1} con innerHTML limpio: '{inner_html}'")

driver.quit()

Elemento 1 con innerHTML limpio: 'Glaze (3ml)'
Elemento 2 con innerHTML limpio: 'Icing (3ml)'
Elemento 3 con innerHTML limpio: 'Drizzle (3ml)'


In [12]:
url = "https://www.sephora.es/p/beautyblender---esponja-de-maquillaje-P10060347.html"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

formatos = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "variation-title.bidirectional"))).text

print(formatos)

driver.quit()

Limoncello


In [19]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

driver.set_script_timeout(60)

ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(100):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
        print(f"[{i}] Scroll height: {new_height} - Productos: {productos}")

        if new_height == last_height and i > 5:
            print("No se cargan más productos, saliendo del bucle.")
            break
        last_height = new_height

    except Exception as e:
        print(f"Error al hacer scroll en la iteración {i}: {e}")
        break

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

productos_urls = productos_urls[:10]  # limitar para pruebas
print(f"Total productos: {len(productos_urls)}")

driver.quit()

# Scraping por producto
dictio_scrap = {
    "marca": [], "nombre": [], "descripcion": [], "precio": [],
    "numero_valoraciones": [], "valoracion": [], "variaciones": [],
    "categoria": [], "subcategoria": []
}

driver = webdriver.Chrome(service=service, options=options)

for product_url in productos_urls:
    driver.get(product_url)

    marca = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))
    ).text
    dictio_scrap["marca"].append(marca)

    titulo = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold"))
    )[0].text

    if " - " in titulo:
        nombre, descripcion = titulo.split(" - ")
    else:
        nombre = titulo
        descripcion = "Sin descripción"

    dictio_scrap["nombre"].append(nombre)
    dictio_scrap["descripcion"].append(descripcion)

    precio = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard"))
    )[0].text.replace(" €", "")
    dictio_scrap["precio"].append(precio)

    numero_valoraciones = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline"))
    )[0].text.split(' ')[0]
    dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

    categoria = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element"))
    )[2].text
    dictio_scrap["categoria"].append(categoria)

    subcategoria = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element"))
    )[3].text
    dictio_scrap["subcategoria"].append(subcategoria)

    try:
        contenido = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))
        ).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        dictio_scrap["valoracion"].append(valoracion)
    except TimeoutException:
        dictio_scrap["valoracion"].append(0)
    except Exception:
        dictio_scrap["valoracion"].append(0)

    distintas_variaciones = []

    # Caso con .variation-title span
    try:
        formatos = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".variation-title span"))
        )
        for formato in formatos:
            inner_html = formato.get_attribute("innerHTML").strip()
            distintas_variaciones.append(inner_html)
    except TimeoutException:
        pass

    # Si no hay varias variaciones, intenta con .variation-title.bidirectional
    if not distintas_variaciones:
        try:
            formato = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "variation-title.bidirectional"))
            )
            texto = formato.text.strip()
            distintas_variaciones.append(texto)
        except TimeoutException:
            pass

    dictio_scrap["variaciones"].append(distintas_variaciones)

driver.quit()

print(dictio_scrap)

[0] Scroll height: 16344 - Productos: 72
[1] Scroll height: 23784 - Productos: 120
[2] Scroll height: 31224 - Productos: 168
[3] Scroll height: 38664 - Productos: 216
[4] Scroll height: 46104 - Productos: 264
[5] Scroll height: 53544 - Productos: 312
[6] Scroll height: 60984 - Productos: 360
[7] Scroll height: 68424 - Productos: 408
[8] Scroll height: 75864 - Productos: 456
[9] Scroll height: 83304 - Productos: 504
[10] Scroll height: 90744 - Productos: 552
[11] Scroll height: 98184 - Productos: 600
[12] Scroll height: 105624 - Productos: 648
[13] Scroll height: 113064 - Productos: 696
[14] Scroll height: 120504 - Productos: 744
[15] Scroll height: 127944 - Productos: 792
[16] Scroll height: 135384 - Productos: 840
[17] Scroll height: 142824 - Productos: 888
[18] Scroll height: 150264 - Productos: 936
[19] Scroll height: 157704 - Productos: 984
[20] Scroll height: 165144 - Productos: 1032
[21] Scroll height: 172584 - Productos: 1080
[22] Scroll height: 180024 - Productos: 1128
[23] Scr

In [17]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

driver.set_script_timeout(90)

ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(100):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
        print(f"[{i}] Scroll height: {new_height} - Productos: {productos}")

        if new_height == last_height and i > 5:
            print("No se cargan más productos, saliendo del bucle.")
            break
        last_height = new_height

    except Exception as e:
        print(f"Error al hacer scroll en la iteración {i}: {e}")
        break

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []

for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    product_data = json.loads(data_tcproduct)
    product_url = product_data.get('product_url_page')
    product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
    productos_urls.append(product_url)

productos_urls = productos_urls[:30]  
print(f"Total productos: {len(productos_urls)}")

driver.quit()

dictio_scrap = {"marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": [], "valoracion": [], "num_variaciones": [], "categoria": [], "subcategoria": []}

driver = webdriver.Chrome(service=service, options=options)

for product_url in productos_urls:
    driver.get(product_url)

    marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text
    dictio_scrap["marca"].append(marca)

    titulo = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text

    if " - " in titulo:
        nombre, descripcion = titulo.split(" - ")
    else:
        nombre = titulo
        descripcion = "Sin descripción"

    dictio_scrap["nombre"].append(nombre)
    dictio_scrap["descripcion"].append(descripcion)

    precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")
    dictio_scrap["precio"].append(precio)

    numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]
    dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

    categoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[2].text
    dictio_scrap["categoria"].append(categoria)

    subcategoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[3].text
    dictio_scrap["subcategoria"].append(subcategoria)

    try:
        contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        dictio_scrap["valoracion"].append(valoracion)
    except TimeoutException:
        dictio_scrap["valoracion"].append(0)
    except Exception:
        dictio_scrap["valoracion"].append(0)

    variaciones = "No tiene variaciones en el producto"

    try:
        tonos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = tonos 
    except Exception as e:
        pass

    try:
        tamaños = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = tamaños 
    except Exception as e:
        pass

    dictio_scrap["num_variaciones"].append(variaciones)

driver.quit()

print(dictio_scrap)

[0] Scroll height: 16344 - Productos: 72
[1] Scroll height: 23784 - Productos: 120
[2] Scroll height: 31224 - Productos: 168
[3] Scroll height: 38664 - Productos: 216
[4] Scroll height: 46104 - Productos: 264
[5] Scroll height: 53544 - Productos: 312
[6] Scroll height: 60984 - Productos: 360
[7] Scroll height: 68424 - Productos: 408
[8] Scroll height: 75864 - Productos: 456
[9] Scroll height: 83304 - Productos: 504
[10] Scroll height: 90744 - Productos: 552
[11] Scroll height: 98184 - Productos: 600
[12] Scroll height: 105624 - Productos: 648
[13] Scroll height: 113064 - Productos: 696
[14] Scroll height: 120504 - Productos: 744
[15] Scroll height: 127944 - Productos: 792
[16] Scroll height: 135384 - Productos: 840
[17] Scroll height: 142824 - Productos: 888
[18] Scroll height: 150264 - Productos: 936
[19] Scroll height: 157704 - Productos: 984
[20] Scroll height: 165144 - Productos: 1032
[21] Scroll height: 172584 - Productos: 1080
[22] Scroll height: 180024 - Productos: 1128
[23] Scr

In [18]:
df = pd.DataFrame(dictio_scrap)
df

,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,categoria,subcategoria
0,Tarte,maracuja juicy melt mask,Mascarilla labial,"27,00",0,0,3,Labios,Bálsamo labial
1,Valentino Make Up,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...","39,99",1,4.0,5,Labios,Gloss/ Brillo labial
2,Fenty Beauty,Hella Thicc Extra Primer,Base voluminizadora para máscaras de pestañas,"21,00",0,0,No tiene variaciones en el producto,Ojos,Máscara de pestañas
3,Fenty Beauty,Gloss Bomb Oil,Aceite-gloss para los labios,"28,00",0,0,7,Labios,Gloss/ Brillo labial
4,Charlotte Tilbury,Pillow Talk Lip & Cheek Set,Estuche labios y mejillas,"55,00",0,0,2,Estuches maquillaje,Estuches multi-usos
5,Charlotte Tilbury,Pillow Talk Love Efecto Lipstick,Lápiz de labios,"36,00",0,0,4,Labios,Barra de labios
6,Nars,Sweet Kiss Quad Eyeshadow,Paleta de 4 sombras de ojos,"52,00",1,3.0,No tiene variaciones en el producto,Ojos,Paleta de sombras de ojos
7,Nudestix,Nudies All Over Face Color Bronze + Glow,"Para ojos, mejillas y labios","33,99",475,4.4,3,Rostro,Colorete
8,Tarte,Life Lock™ hydrating setting spray,Spray fijador hidratante,"37,00",2,5.0,2,Rostro,Primer & spray fijador
9,Dior,Dior Forever Hydra Nude,Fondo de maquillaje acabado natural hidratació...,"63,00",1,5.0,14,Rostro,Base de maquillaje


In [5]:
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))

filtros_urls = []

for filtro in filtros:
    href = filtro.get_attribute("href")
    filtros_urls.append(href)

print(filtros_urls)

driver.quit()

['https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=discountRange&prefv1=25&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=discountRange&prefv1=35&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Standard&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Set%2FEstuche%2FKit&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Formato%20viaje&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Frasco&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Stick&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora

In [10]:
palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]

url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))

filtros_colores = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'gtmrefinement') and contains(@class, 'swatch') and contains(@class, 'empty') and contains(@class, 'swatch-')]")))

filtros_urls = []

for filtro in filtros:
    href = filtro.get_attribute("href")
    if href and any(palabra in href for palabra in palabras_clave):
        filtros_urls.append(href)

for colores in filtros_colores:
    href = colores.get_attribute("href")
    filtros_urls.append(href)

driver.quit()

['https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Standard&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Set%2FEstuche%2FKit&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Formato%20viaje&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Frasco&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Stick&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Frasco%20recargable%2FVaporizador&srule=Best%20seller%20top%20cat%20%26%20new', 'https://www.sephora.es/todos-los-productos/maquillaje-c302/?prefn1=formats&prefv1=Recarga&srule=Best%20seller%20top%20cat%20%26%20ne

In [11]:
len(filtros_urls)

88

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, json, re

url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)
driver.set_script_timeout(90)

ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(100):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
    print(f"[{i}] Scroll height: {new_height} - Productos: {productos}")

    if new_height == last_height and i > 5:
        break
    last_height = new_height

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []
for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    if data_tcproduct:
        product_data = json.loads(data_tcproduct)
        product_url = product_data.get('product_url_page')
        product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
        productos_urls.append(product_url)

productos_urls = productos_urls[:10]
print(f"Total productos: {len(productos_urls)}")

driver.quit()

palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]
mapa_filtros = {
    "formats": "formato",
    "responsibleBeauty": "responsabilidad",
    "eyeshadowEffects": "efecto_sombra",
    "lipEffects": "efecto_labios",
    "mascaraEffects": "efecto_mascara",
    "typesHairBrushes": "tipo_brocha",
    "formulations": "formulacion",
    "skinTypes": "tipo_piel",
    "covers": "cobertura",
    "finishes": "acabado",
    "texture": "textura"
}

filtros_urls = []
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))
filtros_colores = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'gtmrefinement') and contains(@class, 'swatch') and contains(@class, 'empty') and contains(@class, 'swatch-')]")))

for f in filtros + filtros_colores:
    href = f.get_attribute("href")
    if href and any(p in href for p in palabras_clave):
        filtros_urls.append(href)

driver.quit()

productos_por_filtro = {col: [] for col in mapa_filtros.values()}
driver = webdriver.Chrome(service=service, options=options)

for filtro_url in filtros_urls:
    driver.get(filtro_url)
    time.sleep(3)

    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    try:
        valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
    except:
        valor = "Valor desconocido"

    productos_en_filtro = []
    productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
    for p in productos:
        data = p.get_attribute('data-tcproduct')
        if data:
            prod_data = json.loads(data)
            url_producto = prod_data.get('product_url_page')
            if url_producto:
                url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                productos_en_filtro.append(url_producto)

    for clave, columna in mapa_filtros.items():
        if clave in filtro_url:
            productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
            break

driver.quit()

dictio_scrap = {"marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": [], "valoracion": [], "num_variaciones": [], "categoria": [], "subcategoria": []}
dictio_scrap.update({col: [] for col in productos_por_filtro})

driver = webdriver.Chrome(service=service, options=options)

for product_url in productos_urls:
    driver.get(product_url)

    marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text
    dictio_scrap["marca"].append(marca)

    titulo = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
    nombre, descripcion = titulo.split(" - ") if " - " in titulo else (titulo, "Sin descripción")
    dictio_scrap["nombre"].append(nombre)
    dictio_scrap["descripcion"].append(descripcion)

    precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")
    dictio_scrap["precio"].append(precio)

    numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]
    dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

    categoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[2].text
    subcategoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[3].text
    dictio_scrap["categoria"].append(categoria)
    dictio_scrap["subcategoria"].append(subcategoria)

    try:
        contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        dictio_scrap["valoracion"].append(valoracion)
    except:
        dictio_scrap["valoracion"].append(0)

    variaciones = "No tiene variaciones"
    try:
        tonos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = tonos
    except:
        pass
    try:
        tamanos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = tamanos
    except:
        pass
    dictio_scrap["num_variaciones"].append(variaciones)

    for columna, filtros in productos_por_filtro.items():
        valores_detectados = []
        for filtro in filtros:
            if product_url in filtro["productos"]:
                valores_detectados.append(filtro["valor"])
        dictio_scrap[columna].append(valores_detectados if valores_detectados else "No hay información")

driver.quit()

[0] Scroll height: 16344 - Productos: 72
[1] Scroll height: 23784 - Productos: 120
[2] Scroll height: 31224 - Productos: 168
[3] Scroll height: 38664 - Productos: 216
[4] Scroll height: 46104 - Productos: 264
[5] Scroll height: 53544 - Productos: 312
[6] Scroll height: 60984 - Productos: 360
[7] Scroll height: 68424 - Productos: 408
[8] Scroll height: 75864 - Productos: 456
[9] Scroll height: 83304 - Productos: 504
[10] Scroll height: 90744 - Productos: 552
[11] Scroll height: 98184 - Productos: 600
[12] Scroll height: 105624 - Productos: 648
[13] Scroll height: 113064 - Productos: 696
[14] Scroll height: 120504 - Productos: 744
[15] Scroll height: 127944 - Productos: 792
[16] Scroll height: 135384 - Productos: 840
[17] Scroll height: 142824 - Productos: 888
[18] Scroll height: 150264 - Productos: 936
[19] Scroll height: 157704 - Productos: 984
[20] Scroll height: 165144 - Productos: 1032
[21] Scroll height: 172584 - Productos: 1080
[22] Scroll height: 180024 - Productos: 1128
[23] Scr

In [14]:
df = pd.DataFrame(dictio_scrap)
df

,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,categoria,subcategoria,formato,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,tipo_brocha,formulacion,tipo_piel,cobertura,acabado,textura
0,Tarte,maracuja juicy melt mask,Mascarilla labial,"27,00",0,0,3,Labios,Bálsamo labial,No hay información,No hay información,No hay información,"[Hidratante, Natural]",No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,[Liquido]
1,Valentino Make Up,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...","39,99",1,4.0,5,Labios,Gloss/ Brillo labial,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,[Liquido]
2,Fenty Beauty,Hella Thicc Extra Primer,Base voluminizadora para máscaras de pestañas,"21,00",0,0,No tiene variaciones,Ojos,Máscara de pestañas,[Standard],No hay información,No hay información,No hay información,[Voluminizador],No hay información,No hay información,[Piel normal],No hay información,No hay información,[Polvo compacto]
3,Fenty Beauty,Gloss Bomb Oil,Aceite-gloss para los labios,"28,00",0,0,7,Labios,Gloss/ Brillo labial,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,[Todo tipo de pieles],No hay información,No hay información,[Liquido]
4,Charlotte Tilbury,Pillow Talk Lip & Cheek Set,Estuche labios y mejillas,"55,00",0,0,2,Estuches maquillaje,Estuches multi-usos,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información
5,Charlotte Tilbury,Pillow Talk Love Efecto Lipstick,Lápiz de labios,"36,00",0,0,4,Labios,Barra de labios,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información
6,Nars,Sweet Kiss Quad Eyeshadow,Paleta de 4 sombras de ojos,"52,00",1,3.0,No tiene variaciones,Ojos,Paleta de sombras de ojos,[Set/Estuche/Kit],No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información
7,Nudestix,Nudies All Over Face Color Bronze + Glow,"Para ojos, mejillas y labios","33,99",475,4.4,3,Rostro,Colorete,[Standard],"[Vegano, Clean at Sephora]","[Irisado/Nacarado, Brillante]",No hay información,No hay información,No hay información,No hay información,[Todo tipo de pieles],No hay información,No hay información,No hay información
8,Tarte,Life Lock™ hydrating setting spray,Spray fijador hidratante,"37,00",3,5.0,2,Rostro,Primer & spray fijador,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información
9,Dior,Dior Forever Hydra Nude,Fondo de maquillaje acabado natural hidratació...,"63,00",1,5.0,14,Rostro,Base de maquillaje,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,No hay información,[Todo tipo de pieles],[Ligera],[Brillante],No hay información


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, json, re
import numpy as np

url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)
driver.set_script_timeout(100)

ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(200):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
    print(f"[{i}] Scroll height: {new_height} - Productos: {productos}")

    if new_height == last_height and i > 5:
        break
    last_height = new_height

productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []
for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    if data_tcproduct:
        product_data = json.loads(data_tcproduct)
        product_url = product_data.get('product_url_page')
        product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
        productos_urls.append(product_url)

print(len(productos_urls))

driver.quit()

palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]
mapa_filtros = {
    "formats": "formato",
    "responsibleBeauty": "responsabilidad",
    "eyeshadowEffects": "efecto_sombra",
    "lipEffects": "efecto_labios",
    "mascaraEffects": "efecto_mascara",
    "typesHairBrushes": "tipo_brocha",
    "formulations": "formulacion",
    "skinTypes": "tipo_piel",
    "covers": "cobertura",
    "finishes": "acabado",
    "texture": "textura"
}

filtros_urls = []
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))

for f in filtros:
    href = f.get_attribute("href")
    if href and any(p in href for p in palabras_clave):
        filtros_urls.append(href)

driver.quit()

productos_por_filtro = {col: [] for col in mapa_filtros.values()}
driver = webdriver.Chrome(service=service, options=options)

for filtro_url in filtros_urls:
    driver.get(filtro_url)
    time.sleep(3)

    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    try:
        valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
    except:
        valor = "Valor desconocido"

    productos_en_filtro = []
    productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
    for p in productos:
        data = p.get_attribute('data-tcproduct')
        if data:
            prod_data = json.loads(data)
            url_producto = prod_data.get('product_url_page')
            if url_producto:
                url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                productos_en_filtro.append(url_producto)

    for clave, columna in mapa_filtros.items():
        if clave in filtro_url:
            productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
            break

driver.quit()

dictio_scrap = {"marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": [], "valoracion": [], "num_variaciones": [], "categoria": [], "subcategoria": []}
dictio_scrap.update({col: [] for col in productos_por_filtro})

driver = webdriver.Chrome(service=service, options=options)

for product_url in productos_urls:
    driver.get(product_url)

    marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text
    dictio_scrap["marca"].append(marca)

    titulo = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
    nombre, descripcion = titulo.split(" - ") if " - " in titulo else (titulo, "Sin descripción")
    dictio_scrap["nombre"].append(nombre)
    dictio_scrap["descripcion"].append(descripcion)

    precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")
    dictio_scrap["precio"].append(precio)

    numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]
    dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

    categoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[2].text
    subcategoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[3].text
    dictio_scrap["categoria"].append(categoria)
    dictio_scrap["subcategoria"].append(subcategoria)

    try:
        contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        dictio_scrap["valoracion"].append(valoracion)
    except:
        dictio_scrap["valoracion"].append(0)

    try:
        tonos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = tonos
    except:
        pass
    try:
        tamanos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = tamanos
    except:
        pass
    dictio_scrap["num_variaciones"].append(0)

    for columna, filtros in productos_por_filtro.items():
        valores_detectados = []
        for filtro in filtros:
            if product_url in filtro["productos"]:
                valores_detectados.append(filtro["valor"])
        dictio_scrap[columna].append(", ".join(valores_detectados) if valores_detectados else np.nan)

driver.quit()

[0] Scroll height: 16344 - Productos: 72
[1] Scroll height: 23784 - Productos: 120
[2] Scroll height: 31224 - Productos: 168
[3] Scroll height: 38664 - Productos: 216
[4] Scroll height: 46104 - Productos: 264
[5] Scroll height: 53544 - Productos: 312
[6] Scroll height: 60984 - Productos: 360
[7] Scroll height: 68424 - Productos: 408
[8] Scroll height: 68568 - Productos: 408
[9] Scroll height: 75864 - Productos: 456
[10] Scroll height: 83304 - Productos: 504
[11] Scroll height: 90744 - Productos: 552
[12] Scroll height: 98184 - Productos: 600
[13] Scroll height: 105624 - Productos: 648
[14] Scroll height: 113064 - Productos: 696
[15] Scroll height: 120504 - Productos: 744
[16] Scroll height: 127944 - Productos: 792
[17] Scroll height: 135384 - Productos: 840
[18] Scroll height: 142824 - Productos: 888
[19] Scroll height: 150264 - Productos: 936
[20] Scroll height: 157704 - Productos: 984
[21] Scroll height: 165144 - Productos: 1032
[22] Scroll height: 172584 - Productos: 1080
[23] Scrol

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, json, re
import numpy as np

url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"

# Configuración del navegador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

# --- Scraping de página principal ---
driver.get(url)
driver.set_script_timeout(100)

# Click en "Ver más"
ver_mas_btn = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]"))
)
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

# Scroll infinito
last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(200):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
    print(f"[Principal {i}] Scroll height: {new_height} - Productos: {productos}")

    if new_height == last_height and i > 5:
        break
    last_height = new_height

# Recoger URLs
productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))

productos_urls = []
for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    if data_tcproduct:
        product_data = json.loads(data_tcproduct)
        product_url = product_data.get('product_url_page')
        product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
        productos_urls.append(product_url)

print(f"\n✅ Total productos recogidos en la página principal: {len(productos_urls)}")

driver.quit()

# --- Filtros ---
palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]
mapa_filtros = {
    "formats": "formato",
    "responsibleBeauty": "responsabilidad",
    "eyeshadowEffects": "efecto_sombra",
    "lipEffects": "efecto_labios",
    "mascaraEffects": "efecto_mascara",
    "typesHairBrushes": "tipo_brocha",
    "formulations": "formulacion",
    "skinTypes": "tipo_piel",
    "covers": "cobertura",
    "finishes": "acabado",
    "texture": "textura"
}

filtros_urls = []
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))

for f in filtros:
    href = f.get_attribute("href")
    if href and any(p in href for p in palabras_clave):
        filtros_urls.append(href)

driver.quit()

productos_por_filtro = {col: [] for col in mapa_filtros.values()}
driver = webdriver.Chrome(service=service, options=options)

for filtro_url in filtros_urls:
    driver.get(filtro_url)
    time.sleep(3)

    # Scroll en filtros (igual que el principal)
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(200):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height and i > 5:
            break
        last_height = new_height

    try:
        valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
    except:
        valor = "Valor desconocido"

    productos_en_filtro = []
    productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
    for p in productos:
        data = p.get_attribute('data-tcproduct')
        if data:
            prod_data = json.loads(data)
            url_producto = prod_data.get('product_url_page')
            if url_producto:
                url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                productos_en_filtro.append(url_producto)

    for clave, columna in mapa_filtros.items():
        if clave in filtro_url:
            productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
            print(f"🔹 Filtro: {columna} | Valor: {valor} | Productos: {len(productos_en_filtro)}")
            break

driver.quit()

# --- Scraping final ---
dictio_scrap = {
    "marca": [], "nombre": [], "descripcion": [], "precio": [], "numero_valoraciones": [],
    "valoracion": [], "num_variaciones": [], "categoria": [], "subcategoria": []
}
dictio_scrap.update({col: [] for col in productos_por_filtro})

driver = webdriver.Chrome(service=service, options=options)

total = len(productos_urls)
for idx, product_url in enumerate(productos_urls, 1):
    print(f"🧪 ({idx}/{total}) Scrapeando: {product_url}")
    try:
        driver.get(product_url)

        marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text
        dictio_scrap["marca"].append(marca)

        titulo = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
        nombre, descripcion = titulo.split(" - ") if " - " in titulo else (titulo, "Sin descripción")
        dictio_scrap["nombre"].append(nombre)
        dictio_scrap["descripcion"].append(descripcion)

        precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")
        dictio_scrap["precio"].append(precio)

        numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]
        dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

        categoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[2].text
        subcategoria = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element")))[3].text
        dictio_scrap["categoria"].append(categoria)
        dictio_scrap["subcategoria"].append(subcategoria)

        try:
            contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
            valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
            dictio_scrap["valoracion"].append(valoracion)
        except:
            dictio_scrap["valoracion"].append(0)

        variaciones = 0
        try:
            tonos = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text
            variaciones = int(re.search(r"\((\d+)\)", tonos).group(1))
        except:
            pass
        try:
            tamanos = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text
            variaciones = int(re.search(r"\((\d+)\)", tamanos).group(1))
        except:
            pass
        dictio_scrap["num_variaciones"].append(variaciones)

        for columna, filtros in productos_por_filtro.items():
            valores_detectados = []
            for filtro in filtros:
                if product_url in filtro["productos"]:
                    valores_detectados.append(filtro["valor"])
            dictio_scrap[columna].append(", ".join(valores_detectados) if valores_detectados else np.nan)

    except Exception as e:
        print(f"❌ Error con producto {product_url}: {e}")
        for key in dictio_scrap:
            dictio_scrap[key].append(np.nan)

driver.quit()

[Principal 0] Scroll height: 16344 - Productos: 72
[Principal 1] Scroll height: 23784 - Productos: 120
[Principal 2] Scroll height: 31224 - Productos: 168
[Principal 3] Scroll height: 38664 - Productos: 216
[Principal 4] Scroll height: 46104 - Productos: 264
[Principal 5] Scroll height: 53544 - Productos: 312
[Principal 6] Scroll height: 60984 - Productos: 360
[Principal 7] Scroll height: 68424 - Productos: 408
[Principal 8] Scroll height: 75864 - Productos: 456
[Principal 9] Scroll height: 83304 - Productos: 504
[Principal 10] Scroll height: 90744 - Productos: 552
[Principal 11] Scroll height: 98184 - Productos: 600
[Principal 12] Scroll height: 105624 - Productos: 648
[Principal 13] Scroll height: 113064 - Productos: 696
[Principal 14] Scroll height: 120504 - Productos: 744
[Principal 15] Scroll height: 127944 - Productos: 792
[Principal 16] Scroll height: 135384 - Productos: 840
[Principal 17] Scroll height: 142824 - Productos: 888
[Principal 18] Scroll height: 150264 - Productos: 9

KeyboardInterrupt: 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, json, re
import numpy as np
import pandas as pd

# Config inicial
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")

# Driver principal
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
driver.set_script_timeout(100)

# Botón "Ver más"
ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

# Scroll profundo
last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(200):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
    print(f"[Principal Scroll {i}] Scroll height: {new_height} - Productos: {productos}")
    if new_height == last_height and i > 5:
        break
    last_height = new_height

# Recolectar URLs de producto
productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))
productos_urls = []
for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    if data_tcproduct:
        product_data = json.loads(data_tcproduct)
        product_url = product_data.get('product_url_page')
        product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
        productos_urls.append(product_url)

productos_urls = productos_urls[:500]
print(f"\n🟢 Productos en página principal: {len(productos_urls)}")
driver.quit()

# Filtros
palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]
mapa_filtros = {
    "formats": "formato",
    "responsibleBeauty": "responsabilidad",
    "eyeshadowEffects": "efecto_sombra",
    "lipEffects": "efecto_labios",
    "mascaraEffects": "efecto_mascara",
    "typesHairBrushes": "tipo_brocha",
    "formulations": "formulacion",
    "skinTypes": "tipo_piel",
    "covers": "cobertura",
    "finishes": "acabado",
    "texture": "textura"
}

filtros_urls = []
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))
for f in filtros:
    href = f.get_attribute("href")
    if href and any(p in href for p in palabras_clave):
        filtros_urls.append(href)

driver.quit()

# Scrapeo de filtros con scroll y botón ver más
productos_por_filtro = {col: [] for col in mapa_filtros.values()}
driver = webdriver.Chrome(service=service, options=options)

for filtro_url in filtros_urls:
    driver.get(filtro_url)
    time.sleep(3)

    # Botón "Ver más" si aparece
    try:
        ver_mas_btn = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
        time.sleep(2)
        driver.execute_script("arguments[0].click();", ver_mas_btn)
        print("🔘 Botón 'Ver más' clicado en filtro.")
    except:
        print("ℹ️ No hay botón 'Ver más' en este filtro.")

    # Scroll profundo en filtro
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(200):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height and i > 5:
            break
        last_height = new_height

    try:
        valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
    except:
        valor = "Valor desconocido"

    productos_en_filtro = []
    productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
    for p in productos:
        data = p.get_attribute('data-tcproduct')
        if data:
            prod_data = json.loads(data)
            url_producto = prod_data.get('product_url_page')
            if url_producto:
                url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                productos_en_filtro.append(url_producto)

    for clave, columna in mapa_filtros.items():
        if clave in filtro_url:
            productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
            print(f"🔹 Filtro: {columna} | Valor: {valor} | Productos: {len(productos_en_filtro)}")
            break

driver.quit()

# Scrapeo final de detalles de productos
dictio_scrap = {
    "marca": [], "nombre": [], "descripcion": [], "precio": [],
    "numero_valoraciones": [], "valoracion": [], "num_variaciones": [],
    "categoria": [], "subcategoria": []
}
dictio_scrap.update({col: [] for col in productos_por_filtro})

driver = webdriver.Chrome(service=service, options=options)

for idx, product_url in enumerate(productos_urls):
    print(f"📦 Procesando producto {idx+1} de {len(productos_urls)}: {product_url}")
    driver.get(product_url)

    try:
        breadcrumb_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element"))
        )
        if len(breadcrumb_elements) < 2 or breadcrumb_elements[1].text.strip() != "Maquillaje":
            print(f"⛔ Producto fuera de 'Maquillaje'. Saltando: {product_url}")
            continue
    except:
        print(f"⚠️ No se pudo obtener breadcrumb del producto: {product_url}")
        continue

    categoria = breadcrumb_elements[2].text.strip()
    if len(breadcrumb_elements) > 3:
        subcategoria = breadcrumb_elements[3].text.strip()
    else:
        subcategoria = np.nan
    dictio_scrap["categoria"].append(categoria)
    dictio_scrap["subcategoria"].append(subcategoria)

    marca = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text
    dictio_scrap["marca"].append(marca)

    titulo = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
    nombre, descripcion = titulo.split(" - ") if " - " in titulo else (titulo, np.nan)
    dictio_scrap["nombre"].append(nombre)
    dictio_scrap["descripcion"].append(descripcion)

    precio = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")
    dictio_scrap["precio"].append(precio)

    numero_valoraciones = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]
    dictio_scrap["numero_valoraciones"].append(numero_valoraciones)

    try:
        contenido = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        dictio_scrap["valoracion"].append(valoracion)
    except:
        dictio_scrap["valoracion"].append(0)

    variaciones = 0
    try:
        tonos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = tonos
    except:
        pass
    try:
        tamanos = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = tamanos
    except:
        pass
    dictio_scrap["num_variaciones"].append(variaciones)

    for columna, filtros in productos_por_filtro.items():
        valores_detectados = []
        for filtro in filtros:
            if product_url in filtro["productos"]:
                valores_detectados.append(filtro["valor"])
        dictio_scrap[columna].append(", ".join(valores_detectados) if valores_detectados else np.nan)

driver.quit()

df = pd.DataFrame(dictio_scrap)
df.to_csv("productos_maquillaje.csv", index=False)

print("✅ CSV guardado como productos_maquillaje.csv")

[Principal Scroll 0] Scroll height: 16344 - Productos: 72
[Principal Scroll 1] Scroll height: 23784 - Productos: 120
[Principal Scroll 2] Scroll height: 31224 - Productos: 168
[Principal Scroll 3] Scroll height: 38664 - Productos: 216
[Principal Scroll 4] Scroll height: 46104 - Productos: 264
[Principal Scroll 5] Scroll height: 53544 - Productos: 312
[Principal Scroll 6] Scroll height: 60984 - Productos: 360
[Principal Scroll 7] Scroll height: 68424 - Productos: 408
[Principal Scroll 8] Scroll height: 75864 - Productos: 456
[Principal Scroll 9] Scroll height: 83304 - Productos: 504
[Principal Scroll 10] Scroll height: 90744 - Productos: 552
[Principal Scroll 11] Scroll height: 98184 - Productos: 600
[Principal Scroll 12] Scroll height: 105624 - Productos: 648
[Principal Scroll 13] Scroll height: 113064 - Productos: 696
[Principal Scroll 14] Scroll height: 120504 - Productos: 744
[Principal Scroll 15] Scroll height: 127944 - Productos: 792
[Principal Scroll 16] Scroll height: 135384 - P

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x008280E3+60707]
	GetHandleVerifier [0x00828124+60772]
	(No symbol) [0x006504FE]
	(No symbol) [0x0068B898]
	(No symbol) [0x006BCF06]
	(No symbol) [0x006B89D5]
	(No symbol) [0x006B7F66]
	(No symbol) [0x006236E5]
	(No symbol) [0x00623C3E]
	(No symbol) [0x006240CD]
	GetHandleVerifier [0x00A6BBC3+2435075]
	GetHandleVerifier [0x00A67163+2416035]
	GetHandleVerifier [0x00A8350C+2531660]
	GetHandleVerifier [0x0083F1B5+155125]
	GetHandleVerifier [0x00845B5D+182173]
	(No symbol) [0x006233B0]
	(No symbol) [0x00622BC3]
	GetHandleVerifier [0x00B8D2AC+3620588]
	BaseThreadInitThunk [0x771C5D49+25]
	RtlInitializeExceptionChain [0x77E6CE3B+107]
	RtlGetAppContainerNamedObjectPath [0x77E6CDC1+561]


In [3]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, json, re
import numpy as np
import pandas as pd

# Config inicial
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")

# Driver principal
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
driver.set_script_timeout(100)

# Botón "Ver más"
ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

# Scroll profundo
last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(200):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
    print(f"[Principal Scroll {i}] Scroll height: {new_height} - Productos: {productos}")
    if new_height == last_height and i > 5:
        break
    last_height = new_height

# Recolectar URLs de producto
productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))
productos_urls = []
for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    if data_tcproduct:
        product_data = json.loads(data_tcproduct)
        product_url = product_data.get('product_url_page')
        product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
        productos_urls.append(product_url)

productos_urls = productos_urls[:50]
print(f"\n🟢 Productos en página principal: {len(productos_urls)}")
driver.quit()

# Filtros
palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]
mapa_filtros = {
    "formats": "formato",
    "responsibleBeauty": "responsabilidad",
    "eyeshadowEffects": "efecto_sombra",
    "lipEffects": "efecto_labios",
    "mascaraEffects": "efecto_mascara",
    "typesHairBrushes": "tipo_brocha",
    "formulations": "formulacion",
    "skinTypes": "tipo_piel",
    "covers": "cobertura",
    "finishes": "acabado",
    "texture": "textura"
}

filtros_urls = []
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))
for f in filtros:
    href = f.get_attribute("href")
    if href and any(p in href for p in palabras_clave):
        filtros_urls.append(href)

driver.quit()

# Scrapeo de filtros con scroll y botón ver más
productos_por_filtro = {col: [] for col in mapa_filtros.values()}
driver = webdriver.Chrome(service=service, options=options)

for filtro_url in filtros_urls:
    driver.get(filtro_url)
    time.sleep(3)

    try:
        ver_mas_btn = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
        time.sleep(2)
        driver.execute_script("arguments[0].click();", ver_mas_btn)
        print("🔘 Botón 'Ver más' clicado en filtro.")
    except:
        print("ℹ️ No hay botón 'Ver más' en este filtro.")

    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(200):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height and i > 5:
            break
        last_height = new_height

    try:
        valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
    except:
        valor = "Valor desconocido"

    productos_en_filtro = []
    productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
    for p in productos:
        data = p.get_attribute('data-tcproduct')
        if data:
            prod_data = json.loads(data)
            url_producto = prod_data.get('product_url_page')
            if url_producto:
                url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                productos_en_filtro.append(url_producto)

    for clave, columna in mapa_filtros.items():
        if clave in filtro_url:
            productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
            print(f"🔹 Filtro: {columna} | Valor: {valor} | Productos: {len(productos_en_filtro)}")
            break

driver.quit()

# Scrapeo final usando list_scrap
list_scrap = []

driver = webdriver.Chrome(service=service, options=options)

for idx, product_url in enumerate(productos_urls):
    print(f"📦 Procesando producto {idx+1} de {len(productos_urls)}: {product_url}")
    driver.get(product_url)

    try:
        breadcrumb_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element"))
        )
        if len(breadcrumb_elements) < 2 or breadcrumb_elements[1].text.strip() != "Maquillaje":
            print(f"⛔ Producto fuera de 'Maquillaje'. Saltando: {product_url}")
            continue
    except:
        print(f"⚠️ No se pudo obtener breadcrumb del producto: {product_url}")
        continue

    producto_info = {}

    producto_info["categoria"] = breadcrumb_elements[2].text.strip()
    producto_info["subcategoria"] = breadcrumb_elements[3].text.strip() if len(breadcrumb_elements) > 3 else np.nan

    producto_info["marca"] = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text

    titulo = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
    nombre, descripcion = titulo.split(" - ") if " - " in titulo else (titulo, np.nan)
    producto_info["nombre"] = nombre
    producto_info["descripcion"] = descripcion

    producto_info["precio"] = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text.replace(" €", "")

    producto_info["numero_valoraciones"] = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "bv_numReviews_text.underline")))[0].text.split(' ')[0]

    try:
        contenido = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        producto_info["valoracion"] = valoracion
    except:
        producto_info["valoracion"] = 0

    variaciones = 0
    try:
        tonos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = tonos
    except:
        pass
    try:
        tamanos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = tamanos
    except:
        pass
    producto_info["num_variaciones"] = variaciones

    for columna, filtros in productos_por_filtro.items():
        valores_detectados = []
        for filtro in filtros:
            if product_url in filtro["productos"]:
                valores_detectados.append(filtro["valor"])
        producto_info[columna] = ", ".join(valores_detectados) if valores_detectados else np.nan

    list_scrap.append(producto_info)

driver.quit()

# Pasar lista a DataFrame y guardar
df = pd.DataFrame(list_scrap)
df.to_csv("productos_maquillaje.csv", index=False)
print("✅ CSV guardado como productos_maquillaje.csv")

[Principal Scroll 0] Scroll height: 16344 - Productos: 72
[Principal Scroll 1] Scroll height: 23784 - Productos: 120
[Principal Scroll 2] Scroll height: 31224 - Productos: 168
[Principal Scroll 3] Scroll height: 38664 - Productos: 216
[Principal Scroll 4] Scroll height: 46104 - Productos: 264
[Principal Scroll 5] Scroll height: 53544 - Productos: 312
[Principal Scroll 6] Scroll height: 60984 - Productos: 360
[Principal Scroll 7] Scroll height: 68424 - Productos: 408
[Principal Scroll 8] Scroll height: 75864 - Productos: 456
[Principal Scroll 9] Scroll height: 83304 - Productos: 504
[Principal Scroll 10] Scroll height: 90744 - Productos: 552
[Principal Scroll 11] Scroll height: 98184 - Productos: 600
[Principal Scroll 12] Scroll height: 105624 - Productos: 648
[Principal Scroll 13] Scroll height: 113064 - Productos: 696
[Principal Scroll 14] Scroll height: 120504 - Productos: 744
[Principal Scroll 15] Scroll height: 127944 - Productos: 792
[Principal Scroll 16] Scroll height: 135384 - P

In [5]:
df.head()

,categoria,subcategoria,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,formato,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,tipo_brocha,formulacion,tipo_piel,cobertura,acabado,textura
0,Ojos,Sombras de ojos,Rem Beauty,Starlet,Sombra de ojos líquida,"23,00",0,2.0,12,NaN,NaN,"Mate, Irisado/Nacarado, Brillante",NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,NaN,Liquido
1,Rostro,Polvos bronceadores,Dior,Dior Forever Nude Bronze,Polvos bronceadores acabado radiante edición l...,"65,00",0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,Ligera,Natural,NaN
2,Labios,Voluminizador labios,Dior Backstage,Dior Lip Glow Oil,Aceite para labios brillante nutritivo realzad...,"48,00",0,4.2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,NaN,Aceite
3,Ojos,Paleta de sombras de ojos,Dior,Diorshow 5 Couleurs edición limitada,sombras de ojos de colores intensos,"75,00",0,0,2,NaN,NaN,Irisado/Nacarado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Uñas,Esmalte de uñas,Dior,Dior Vernis,Laca de uñas efecto gel color couture,"37,00",0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,Purpurina,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   categoria            46 non-null     object
 1   subcategoria         46 non-null     object
 2   marca                46 non-null     object
 3   nombre               46 non-null     object
 4   descripcion          45 non-null     object
 5   precio               46 non-null     object
 6   numero_valoraciones  46 non-null     object
 7   valoracion           46 non-null     object
 8   num_variaciones      46 non-null     object
 9   formato              21 non-null     object
 10  responsabilidad      13 non-null     object
 11  efecto_sombra        6 non-null      object
 12  efecto_labios        9 non-null      object
 13  efecto_mascara       4 non-null      object
 14  tipo_brocha          1 non-null      object
 15  formulacion          1 non-null      object
 16  tipo_piel 

In [3]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, json, re
import numpy as np
import pandas as pd
from datetime import date

# Config inicial
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")

# Driver principal
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
driver.set_script_timeout(100)

# Botón "Ver más"
ver_mas_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

# Scroll profundo
last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(200):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
    print(f"[Principal Scroll {i}] Scroll height: {new_height} - Productos: {productos}")
    if new_height == last_height and i > 5:
        break
    last_height = new_height

# Recolectar URLs de producto
productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))
productos_urls = []
for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    if data_tcproduct:
        product_data = json.loads(data_tcproduct)
        product_url = product_data.get('product_url_page')
        product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
        productos_urls.append(product_url)

productos_urls = productos_urls[:5]
print(f"\n🟢 Productos en página principal: {len(productos_urls)}")
driver.quit()

# Filtros
palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]
mapa_filtros = {
    "formats": "formato",
    "responsibleBeauty": "responsabilidad",
    "eyeshadowEffects": "efecto_sombra",
    "lipEffects": "efecto_labios",
    "mascaraEffects": "efecto_mascara",
    "typesHairBrushes": "tipo_brocha",
    "formulations": "formulacion",
    "skinTypes": "tipo_piel",
    "covers": "cobertura",
    "finishes": "acabado",
    "texture": "textura"
}

filtros_urls = []
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))
for f in filtros:
    href = f.get_attribute("href")
    if href and any(p in href for p in palabras_clave):
        filtros_urls.append(href)

driver.quit()

# Scrapeo de filtros
productos_por_filtro = {col: [] for col in mapa_filtros.values()}
driver = webdriver.Chrome(service=service, options=options)

for filtro_url in filtros_urls:
    driver.get(filtro_url)
    time.sleep(3)

    try:
        ver_mas_btn = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
        time.sleep(2)
        driver.execute_script("arguments[0].click();", ver_mas_btn)
        print("🔘 Botón 'Ver más' clicado en filtro.")
    except:
        print("ℹ️ No hay botón 'Ver más' en este filtro.")

    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(200):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height and i > 5:
            break
        last_height = new_height

    try:
        valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
    except:
        valor = "Valor desconocido"

    productos_en_filtro = []
    productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
    for p in productos:
        data = p.get_attribute('data-tcproduct')
        if data:
            prod_data = json.loads(data)
            url_producto = prod_data.get('product_url_page')
            if url_producto:
                url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                productos_en_filtro.append(url_producto)

    for clave, columna in mapa_filtros.items():
        if clave in filtro_url:
            productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
            print(f"🔹 Filtro: {columna} | Valor: {valor} | Productos: {len(productos_en_filtro)}")
            break

driver.quit()

# Scrapeo final usando list_scrap
list_scrap = []

driver = webdriver.Chrome(service=service, options=options)

for idx, product_url in enumerate(productos_urls):
    print(f"📦 Procesando producto {idx+1} de {len(productos_urls)}: {product_url}")
    driver.get(product_url)

    try:
        breadcrumb_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element"))
        )
        if len(breadcrumb_elements) < 2 or breadcrumb_elements[1].text.strip() != "Maquillaje":
            print(f"⛔ Producto fuera de 'Maquillaje'. Saltando: {product_url}")
            continue
    except:
        print(f"⚠️ No se pudo obtener breadcrumb del producto: {product_url}")
        continue

    producto_info = {}
    producto_info["categoria"] = breadcrumb_elements[2].text.strip()
    producto_info["subcategoria"] = breadcrumb_elements[3].text.strip() if len(breadcrumb_elements) > 3 else np.nan

    producto_info["marca"] = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text

    titulo = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
    nombre, descripcion = titulo.split(" - ") if " - " in titulo else (titulo, np.nan)
    producto_info["nombre"] = nombre
    producto_info["descripcion"] = descripcion

    try:
        precio = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text
        producto_info["precio"] = float(precio.replace(" €", "").replace(",", "."))
    except:
        producto_info["precio"] = np.nan

    try:
        n_val = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-number-review"))).get_attribute('innerHTML').strip().split(' ')[0]
        producto_info["numero_valoraciones"] = int(n_val)
    except:
        producto_info["numero_valoraciones"] = 0

    try:
        contenido = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        producto_info["valoracion"] = float(valoracion)
    except:
        producto_info["valoracion"] = 0

    variaciones = 0
    try:
        tonos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = int(tonos)
    except:
        pass
    try:
        tamanos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = int(tamanos)
    except:
        pass
    producto_info["num_variaciones"] = variaciones

    producto_info["fecha_extraccion"] = pd.to_datetime(date.today())

    for columna, filtros in productos_por_filtro.items():
        valores_detectados = []
        for filtro in filtros:
            if product_url in filtro["productos"]:
                valores_detectados.append(filtro["valor"])
        producto_info[columna] = ", ".join(valores_detectados) if valores_detectados else None

    list_scrap.append(producto_info)

driver.quit()

df = pd.DataFrame(list_scrap)
hoy = date.today().strftime("%Y-%m-%d")
nombre_archivo = f"productos_maquillaje_{hoy}.csv"
df.to_csv(nombre_archivo, index=False)
print(f"✅ CSV guardado como {nombre_archivo}")

[Principal Scroll 0] Scroll height: 16344 - Productos: 72
[Principal Scroll 1] Scroll height: 23784 - Productos: 120
[Principal Scroll 2] Scroll height: 31224 - Productos: 168
[Principal Scroll 3] Scroll height: 38664 - Productos: 216
[Principal Scroll 4] Scroll height: 46104 - Productos: 264
[Principal Scroll 5] Scroll height: 53544 - Productos: 312
[Principal Scroll 6] Scroll height: 60984 - Productos: 360
[Principal Scroll 7] Scroll height: 68424 - Productos: 408
[Principal Scroll 8] Scroll height: 75864 - Productos: 456
[Principal Scroll 9] Scroll height: 83304 - Productos: 504
[Principal Scroll 10] Scroll height: 90744 - Productos: 552
[Principal Scroll 11] Scroll height: 98184 - Productos: 600
[Principal Scroll 12] Scroll height: 105624 - Productos: 648
[Principal Scroll 13] Scroll height: 113064 - Productos: 696
[Principal Scroll 14] Scroll height: 120504 - Productos: 744
[Principal Scroll 15] Scroll height: 127944 - Productos: 792
[Principal Scroll 16] Scroll height: 135384 - P

In [4]:
df

,categoria,subcategoria,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,fecha_extraccion,...,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,tipo_brocha,formulacion,tipo_piel,cobertura,acabado,textura
0,Ojos,Sombras de ojos,Rem Beauty,Starlet,Sombra de ojos líquida,23.00,1,2.0,12,2025-04-19,...,Clean at Sephora,"Mate, Irisado/Nacarado, Brillante",None,None,None,None,Todo tipo de pieles,None,None,Liquido
1,Labios,Voluminizador labios,Dior Backstage,Dior Lip Glow Oil,Aceite para labios brillante nutritivo realzad...,48.00,437,4.2,3,2025-04-19,...,None,None,None,None,None,None,Todo tipo de pieles,None,None,Aceite
2,Uñas,Esmalte de uñas,Dior,Dior Vernis,Laca de uñas efecto gel color couture,37.00,0,0.0,2,2025-04-19,...,None,None,None,None,None,None,Todo tipo de pieles,None,Purpurina,None
3,Labios,Gloss/ Brillo labial,Valentino Make Up,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...",39.99,5,4.6,5,2025-04-19,...,None,None,None,None,None,None,None,None,None,Liquido
4,Labios,Gloss/ Brillo labial,Fenty Beauty,Gloss Bomb Oil,Aceite-gloss para los labios,28.00,0,0.0,7,2025-04-19,...,None,None,None,None,None,None,Todo tipo de pieles,None,None,Liquido


In [5]:
pd.set_option('display.max_columns', None)

df

,categoria,subcategoria,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,fecha_extraccion,formato,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,tipo_brocha,formulacion,tipo_piel,cobertura,acabado,textura
0,Ojos,Sombras de ojos,Rem Beauty,Starlet,Sombra de ojos líquida,23.0,1,2.0,12,2025-04-15,None,None,"Mate, Irisado/Nacarado, Brillante",None,None,None,None,Todo tipo de pieles,None,None,Liquido
1,Rostro,Polvos bronceadores,Dior,Dior Forever Nude Bronze,Polvos bronceadores acabado radiante edición l...,65.0,0,0.0,2,2025-04-15,None,None,None,None,None,None,None,Todo tipo de pieles,Ligera,Natural,None
2,Labios,Voluminizador labios,Dior Backstage,Dior Lip Glow Oil,Aceite para labios brillante nutritivo realzad...,48.0,434,4.2,3,2025-04-15,None,None,None,None,None,None,None,Todo tipo de pieles,None,None,Aceite
3,Ojos,Paleta de sombras de ojos,Dior,Diorshow 5 Couleurs edición limitada,sombras de ojos de colores intensos,75.0,0,0.0,2,2025-04-15,None,None,Irisado/Nacarado,None,None,None,None,None,None,None,None
4,Uñas,Esmalte de uñas,Dior,Dior Vernis,Laca de uñas efecto gel color couture,37.0,0,0.0,2,2025-04-15,None,None,None,None,None,None,None,Todo tipo de pieles,None,Purpurina,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   categoria            5 non-null      object        
 1   subcategoria         5 non-null      object        
 2   marca                5 non-null      object        
 3   nombre               5 non-null      object        
 4   descripcion          5 non-null      object        
 5   precio               5 non-null      float64       
 6   numero_valoraciones  5 non-null      int64         
 7   valoracion           5 non-null      float64       
 8   num_variaciones      5 non-null      int64         
 9   fecha_extraccion     5 non-null      datetime64[ns]
 10  formato              0 non-null      object        
 11  responsabilidad      0 non-null      object        
 12  efecto_sombra        2 non-null      object        
 13  efecto_labios        0 non-null      ob

In [ ]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, json, re
import numpy as np
import pandas as pd
from datetime import date

# Función para inicializar el driver
def iniciar_driver():
    service = Service(ChromeDriverManager().install())
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Función para hacer scroll y clic en el botón 'Ver más'
def hacer_scroll_y_cargar(driver, boton_ver_mas_xpath=None):
    if boton_ver_mas_xpath:
        try:
            ver_mas_btn = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, boton_ver_mas_xpath))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
            time.sleep(2)
            driver.execute_script("arguments[0].click();", ver_mas_btn)
            print(f"🔘 Botón 'Ver más' clicado.")
        except:
            print("ℹ️ No hay botón 'Ver más'.")

    # Scroll profundo
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(200):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height and i > 5:
            break
        last_height = new_height

# Función para obtener los productos de la página principal
def obtener_productos_principales(driver, url):
    driver.get(url)
    hacer_scroll_y_cargar(driver, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")
    
    productos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile")))
    productos_urls = []
    for producto in productos:
        data_tcproduct = producto.get_attribute('data-tcproduct')
        if data_tcproduct:
            product_data = json.loads(data_tcproduct)
            product_url = product_data.get('product_url_page')
            product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
            productos_urls.append(product_url)

    return productos_urls

# Función para obtener los filtros disponibles
def obtener_filtros(driver, url):
    driver.get(url)
    filtros_urls = []
    filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))
    palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]
    
    for f in filtros:
        href = f.get_attribute("href")
        if href and any(p in href for p in palabras_clave):
            filtros_urls.append(href)

    return filtros_urls

# Función para recolectar productos de los filtros
def obtener_productos_filtros(driver, filtros_urls):
    mapa_filtros = {
    "formats": "formato",
    "responsibleBeauty": "responsable",
    "eyeshadowEffects": "efecto_ojos",
    "lipEffects": "efecto_labios",
    "mascaraEffects": "efecto_mascara",
    "typesHairBrushes": "tipo_cepillo",
    "formulations": "formulacion",
    "skinTypes": "tipo_piel",
    "covers": "cobertura",
    "finishes": "acabado",
    "texture": "textura"
}
    productos_por_filtro = {col: [] for col in mapa_filtros.values()}
    
    for filtro_url in filtros_urls:
        driver.get(filtro_url)
        time.sleep(3)
        hacer_scroll_y_cargar(driver, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]")

        try:
            valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
        except:
            valor = "Valor desconocido"

        productos_en_filtro = []
        productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
        for p in productos:
            data = p.get_attribute('data-tcproduct')
            if data:
                prod_data = json.loads(data)
                url_producto = prod_data.get('product_url_page')
                if url_producto:
                    url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                    productos_en_filtro.append(url_producto)

        for clave, columna in mapa_filtros.items():
            if clave in filtro_url:
                productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
                print(f"🔹 Filtro: {columna} | Valor: {valor} | Productos: {len(productos_en_filtro)}")
                break

    return productos_por_filtro

# Función para recolectar información de un producto individual
def obtener_info_producto(driver, product_url, productos_por_filtro):
    producto_info = {}
    driver.get(product_url)

    try:
        breadcrumb_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element"))
        )
        if len(breadcrumb_elements) < 2 or breadcrumb_elements[1].text.strip() != "Maquillaje":
            print(f"⛔ Producto fuera de 'Maquillaje'. Saltando: {product_url}")
            return None
    except:
        print(f"⚠️ No se pudo obtener breadcrumb del producto: {product_url}")
        return None

    producto_info["categoria"] = breadcrumb_elements[2].text.strip()
    producto_info["subcategoria"] = breadcrumb_elements[3].text.strip() if len(breadcrumb_elements) > 3 else np.nan

    producto_info["marca"] = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text

    titulo = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
    nombre, descripcion = titulo.split(" - ") if " - " in titulo else (titulo, np.nan)
    producto_info["nombre"] = nombre
    producto_info["descripcion"] = descripcion

    try:
        precio = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text
        producto_info["precio"] = float(precio.replace(" €", "").replace(",", "."))
    except:
        producto_info["precio"] = np.nan

    try:
        n_val = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-number-review"))).get_attribute('innerHTML').strip().split(' ')[0]
        producto_info["numero_valoraciones"] = int(n_val)
    except:
        producto_info["numero_valoraciones"] = 0

    try:
        contenido = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        producto_info["valoracion"] = float(valoracion)
    except:
        producto_info["valoracion"] = 0

    variaciones = 0
    try:
        tonos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = int(tonos)
    except:
        pass
    try:
        tamanos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = int(tamanos)
    except:
        pass
    producto_info["num_variaciones"] = variaciones

    producto_info["fecha_extraccion"] = pd.to_datetime(date.today())

    for columna, filtros in productos_por_filtro.items():
        valores_detectados = []
        for filtro in filtros:
            if product_url in filtro["productos"]:
                valores_detectados.append(filtro["valor"])
        producto_info[columna] = ", ".join(valores_detectados) if valores_detectados else None

    return producto_info

# Función principal para scrapear la web
def scrapear_sephora():
    url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"
    
    # Inicializamos driver
    driver = iniciar_driver()

    # Obtenemos productos de la página principal
    productos_urls = obtener_productos_principales(driver, url)

    # Obtenemos los filtros disponibles
    filtros_urls = obtener_filtros(driver, url)

    # Obtenemos los productos por filtro
    productos_por_filtro = obtener_productos_filtros(driver, filtros_urls)

    # Recolectamos la información de los productos
    list_scrap = []
    for idx, product_url in enumerate(productos_urls):
        print(f"📦 Procesando producto {idx+1} de {len(productos_urls)}: {product_url}")
        producto_info = obtener_info_producto(driver, product_url, productos_por_filtro)
        if producto_info:
            list_scrap.append(producto_info)

    # Cerramos el driver
    driver.quit()

    # Crear DataFrame
    df = pd.DataFrame(list_scrap)
    hoy = date.today().strftime("%Y-%m-%d")
    nombre_archivo = f"productos_maquillaje_{hoy}.csv"
    df.to_csv(nombre_archivo, index=False)
    print(f"✅ CSV guardado como {nombre_archivo}")
    
    return df

# Llamada a la función principal
df_resultado = scrapear_sephora()

In [55]:
url = "https://www.sephora.es/p/dior-lip-glow-oil---aceite-para-labios-brillante-nutritivo-realzador-del-color-P1000207243.html"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

producto_info = {}

try:
    n_val = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-number-review"))).get_attribute('innerHTML').strip().split(' ')[0]
    producto_info["numero_valoraciones"] = int(n_val)
except:
    producto_info["numero_valoraciones"] = 0

print(producto_info)

driver.quit()

{'numero_valoraciones': 434}


In [56]:
url = "https://www.sephora.es/p/dior-lip-glow-oil---aceite-para-labios-brillante-nutritivo-realzador-del-color-P1000207243.html"

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

producto_info = {}

try:
    n_val = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv_number-review"))).get_attribute('innerHTML').strip()
    producto_info["numero_valoraciones"] = int(n_val.split(' ')[0])
except:
    producto_info["numero_valoraciones"] = 0

print(producto_info)

driver.quit()

{'numero_valoraciones': 0}


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from datetime import date
import time, json, re, os
import numpy as np
import pandas as pd

# Config inicial
url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")

# Driver principal para scroll y URLs
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
driver.set_script_timeout(100)

# Botón "Ver más"
ver_mas_btn = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]"))
)
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
time.sleep(2)
driver.execute_script("arguments[0].click();", ver_mas_btn)

# Scroll profundo
last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(200):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
    print(f"[Principal Scroll {i}] Scroll height: {new_height} - Productos: {productos}")
    if new_height == last_height and i > 5:
        break
    last_height = new_height

# Recolectar URLs de producto
productos = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile"))
)
productos_urls = []
for producto in productos:
    data_tcproduct = producto.get_attribute('data-tcproduct')
    if data_tcproduct:
        product_data = json.loads(data_tcproduct)
        product_url = product_data.get('product_url_page')
        product_url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', product_url)
        productos_urls.append(product_url)

driver.quit()
print(f"\n🟢 Productos en página principal: {len(productos_urls)}")

# Filtros
palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects",
                 "formulations", "skinTypes", "covers", "finishes", "texture"]
mapa_filtros = {
    "formats": "formato",
    "responsibleBeauty": "responsabilidad",
    "eyeshadowEffects": "efecto_sombra",
    "lipEffects": "efecto_labios",
    "mascaraEffects": "efecto_mascara",
    "formulations": "formulacion",
    "skinTypes": "tipo_piel",
    "covers": "cobertura",
    "finishes": "acabado",
    "texture": "textura"
}

filtros_urls = []
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

filtros = WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item"))
)
for f in filtros:
    href = f.get_attribute("href")
    if href and any(p in href for p in palabras_clave):
        filtros_urls.append(href)

driver.quit()

# Scrapeo de filtros con scroll y botón ver más
productos_por_filtro = {col: [] for col in mapa_filtros.values()}
driver = webdriver.Chrome(service=service, options=options)

for filtro_url in filtros_urls:
    driver.get(filtro_url)
    time.sleep(3)

    try:
        ver_mas_btn = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
        time.sleep(2)
        driver.execute_script("arguments[0].click();", ver_mas_btn)
    except:
        pass

    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(200):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height and i > 5:
            break
        last_height = new_height

    try:
        valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
    except:
        valor = "Valor desconocido"

    productos_en_filtro = []
    productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
    for p in productos:
        data = p.get_attribute('data-tcproduct')
        if data:
            prod_data = json.loads(data)
            url_producto = prod_data.get('product_url_page')
            if url_producto:
                url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                productos_en_filtro.append(url_producto)

    for clave, columna in mapa_filtros.items():
        if clave in filtro_url:
            productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
            break

driver.quit()

# Reanudar desde parcial si existe
parcial_path = "productos_maquillaje_parcial.csv"
if os.path.exists(parcial_path):
    df_parcial = pd.read_csv(parcial_path)
    scraped_urls = set(df_parcial["nombre"].astype(str))
else:
    df_parcial = pd.DataFrame()
    scraped_urls = set()

# Scrapeo final
list_scrap = []
driver = webdriver.Chrome(service=service, options=options)

for idx, product_url in enumerate(productos_urls):
    driver.get(product_url)

    try:
        breadcrumb_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element"))
        )
        if len(breadcrumb_elements) < 2 or breadcrumb_elements[1].text.strip() != "Maquillaje":
            continue
    except:
        continue

    try:
        titulo = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
        nombre, _ = titulo.split(" - ") if " - " in titulo else (titulo, np.nan)
        if nombre in scraped_urls:
            continue
    except:
        continue

    producto_info = {}

    producto_info["categoria"] = breadcrumb_elements[2].text.strip()
    producto_info["subcategoria"] = breadcrumb_elements[3].text.strip() if len(breadcrumb_elements) > 3 else np.nan

    producto_info["marca"] = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text

    producto_info["nombre"] = nombre
    producto_info["descripcion"] = _

    try:
        precio = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text
        producto_info["precio"] = float(precio.replace(" €", "").replace(",", "."))
    except:
        producto_info["precio"] = np.nan

    try:
        n_val = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-number-review"))).get_attribute('innerHTML').strip().split(' ')[0]
        producto_info["numero_valoraciones"] = int(n_val)
    except:
        producto_info["numero_valoraciones"] = 0

    try:
        contenido = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        producto_info["valoracion"] = float(valoracion)
    except:
        producto_info["valoracion"] = 0

    variaciones = 0
    try:
        tonos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = int(tonos)
    except:
        pass
    try:
        tamanos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = int(tamanos)
    except:
        pass
    producto_info["num_variaciones"] = variaciones

    producto_info["fecha_extraccion"] = pd.to_datetime(date.today())

    for columna, filtros in productos_por_filtro.items():
        valores_detectados = []
        for filtro in filtros:
            if product_url in filtro["productos"]:
                valores_detectados.append(filtro["valor"])


        producto_info[columna] = ", ".join(valores_detectados) if valores_detectados else None

    list_scrap.append(producto_info)

    if len(list_scrap) >= 50:
        temp_df = pd.DataFrame(list_scrap)
        df_total = pd.concat([df_parcial, temp_df], ignore_index=True)
        df_total.to_csv(parcial_path, index=False)
        list_scrap = []  # Limpiar la lista para seguir procesando nuevos productos

driver.quit()

# Guardar CSV final
df_total = pd.DataFrame(list_scrap)
df_total.to_csv("productos_maquillaje.csv", index=False)

In [44]:
import pandas as pd
from datetime import datetime

# Fecha de hoy en formato datetime sin hora
fecha_hoy = pd.to_datetime(date.today())

# Crear DataFrame con esa fecha
df = pd.DataFrame({"fecha_extraccion": [fecha_hoy]})

# Ver resultado
print(df)
print(df.dtypes)

  fecha_extraccion
0       2025-04-15
fecha_extraccion    datetime64[ns]
dtype: object


In [6]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, json, re
import numpy as np
import pandas as pd
from datetime import date

def iniciar_driver():
    service = Service(ChromeDriverManager().install())
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_script_timeout(100)
    return driver

def cargar_todos_los_productos(driver, mostrar_productos=True, mostrar_scroll=True):
    try:
        ver_mas_btn = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'see-more-button') and contains(@class, 'secondary-button-revamp')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mas_btn)
        time.sleep(2)
        driver.execute_script("arguments[0].click();", ver_mas_btn)
        if mostrar_productos:
            print("🔘 Botón 'Ver más' clicado.")
    except:
        if mostrar_productos:
            print("ℹ️ No se encontró botón 'Ver más'.")

    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(200):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if mostrar_scroll:
            productos = len(driver.find_elements(By.CLASS_NAME, 'product-brand'))
            print(f"[Scroll {i}] Scroll height: {new_height} - Productos: {productos}")
        if new_height == last_height and i > 5:
            break
        last_height = new_height

def obtener_urls_productos(driver, limite=5):
    productos = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product-tile.clickable.omnibus-tile"))
    )
    urls = []
    for producto in productos:
        data = producto.get_attribute('data-tcproduct')
        if data:
            prod_data = json.loads(data)
            url = prod_data.get('product_url_page')
            url = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url)
            urls.append(url)
    return urls[:limite]

def obtener_urls_filtros(driver, palabras_clave):
    filtros_urls = []
    filtros = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "gtmrefinement.empty.refinement-item")))
    for f in filtros:
        href = f.get_attribute("href")
        if href and any(p in href for p in palabras_clave):
            filtros_urls.append(href)
    return filtros_urls

def scrapeo_filtros(driver, filtros_urls, mapa_filtros):
    productos_por_filtro = {col: [] for col in mapa_filtros.values()}
    for filtro_url in filtros_urls:
        driver.get(filtro_url)
        time.sleep(3)
        cargar_todos_los_productos(driver, mostrar_scroll=False)

        try:
            valor = driver.find_element(By.CLASS_NAME, "breadcrumb-refinement-value").text.strip()
        except:
            valor = "Valor desconocido"

        productos_en_filtro = []
        productos = driver.find_elements(By.CLASS_NAME, "product-tile.clickable.omnibus-tile")
        for p in productos:
            data = p.get_attribute('data-tcproduct')
            if data:
                prod_data = json.loads(data)
                url_producto = prod_data.get('product_url_page')
                if url_producto:
                    url_producto = re.sub(r'-p(\d+)\.html$', r'-P\1.html', url_producto)
                    productos_en_filtro.append(url_producto)

        for clave, columna in mapa_filtros.items():
            if clave in filtro_url:
                productos_por_filtro[columna].append({"valor": valor, "productos": productos_en_filtro})
                print(f"🔹 Filtro: {columna} | Valor: {valor} | Productos: {len(productos_en_filtro)}")
                break

    return productos_por_filtro

def scrapeo_producto(driver, product_url, productos_por_filtro):
    producto_info = {}
    driver.get(product_url)

    try:
        breadcrumb_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "breadcrumb-element"))
        )
        if len(breadcrumb_elements) < 2 or breadcrumb_elements[1].text.strip() != "Maquillaje":
            print(f"⛔ Producto fuera de 'Maquillaje'. Saltando: {product_url}")
            return None
    except:
        print(f"⚠️ No se pudo obtener breadcrumb del producto: {product_url}")
        return None

    producto_info["categoria"] = breadcrumb_elements[2].text.strip()
    producto_info["subcategoria"] = breadcrumb_elements[3].text.strip() if len(breadcrumb_elements) > 3 else np.nan
    producto_info["marca"] = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "brand-name"))).text

    titulo = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-name.product-name-bold")))[0].text
    nombre, descripcion = titulo.split(" - ") if " - " in titulo else (titulo, np.nan)
    producto_info["nombre"] = nombre
    producto_info["descripcion"] = descripcion

    try:
        precio = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "price-sales.price-sales-standard")))[0].text
        producto_info["precio"] = float(precio.replace(" €", "").replace(",", "."))
    except:
        producto_info["precio"] = np.nan

    try:
        n_val = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-number-review"))).get_attribute('innerHTML').strip().split(' ')[0]
        producto_info["numero_valoraciones"] = int(n_val)
    except:
        producto_info["numero_valoraciones"] = 0

    try:
        contenido = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "bv-overall-score"))).get_attribute('innerHTML')
        valoracion = re.search(r"\d+\.\d+/5", contenido).group(0).split("/")[0]
        producto_info["valoracion"] = float(valoracion)
    except:
        producto_info["valoracion"] = 0

    variaciones = 0
    try:
        tonos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-colorguide"))).text.split('(')[1].split(')')[0]
        variaciones = int(tonos)
    except:
        pass
    try:
        tamanos = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CLASS_NAME, "open-selector"))).text.split('(')[1].split(')')[0]
        variaciones = int(tamanos)
    except:
        pass
    producto_info["num_variaciones"] = variaciones

    producto_info["fecha_extraccion"] = pd.to_datetime(date.today())

    for columna, filtros in productos_por_filtro.items():
        valores_detectados = []
        for filtro in filtros:
            if product_url in filtro["productos"]:
                valores_detectados.append(filtro["valor"])
        producto_info[columna] = ", ".join(valores_detectados) if valores_detectados else None

    return producto_info

def scrapeo_sephora():
    url = "https://www.sephora.es/todos-los-productos/maquillaje-c302/"
    palabras_clave = ["formats", "responsibleBeauty", "eyeshadowEffects", "lipEffects", "mascaraEffects", "typesHairBrushes", "formulations", "skinTypes", "covers", "finishes", "texture"]
    mapa_filtros = {
        "formats": "formato",
        "responsibleBeauty": "responsabilidad",
        "eyeshadowEffects": "efecto_sombra",
        "lipEffects": "efecto_labios",
        "mascaraEffects": "efecto_mascara",
        "typesHairBrushes": "tipo_brocha",
        "formulations": "formulacion",
        "skinTypes": "tipo_piel",
        "covers": "cobertura",
        "finishes": "acabado",
        "texture": "textura"
    }

    driver = iniciar_driver()
    driver.get(url)
    cargar_todos_los_productos(driver)
    productos_urls = obtener_urls_productos(driver)
    print(f"\n🟢 Productos en página principal: {len(productos_urls)}")
    driver.quit()

    driver = iniciar_driver()
    driver.get(url)
    filtros_urls = obtener_urls_filtros(driver, palabras_clave)
    print(f"\n🔍 Filtros encontrados: {len(filtros_urls)}")
    driver.quit()

    driver = iniciar_driver()
    productos_por_filtro = scrapeo_filtros(driver, filtros_urls, mapa_filtros)
    driver.quit()

    list_scrap = []
    driver = iniciar_driver()
    for idx, product_url in enumerate(productos_urls):
        print(f"📦 Procesando producto {idx+1} de {len(productos_urls)}: {product_url}")
        info = scrapeo_producto(driver, product_url, productos_por_filtro)
        if info:
            list_scrap.append(info)
    driver.quit()

    df = pd.DataFrame(list_scrap)
    hoy = date.today().strftime("%Y-%m-%d")
    nombre_archivo = f"productos_maquillaje_{hoy}.csv"
    df.to_csv(nombre_archivo, index=False)
    print(f"✅ CSV guardado como {nombre_archivo}")

    return df

In [7]:
df = scrapeo_sephora()
df

🔘 Botón 'Ver más' clicado.
[Scroll 0] Scroll height: 16344 - Productos: 72
[Scroll 1] Scroll height: 23784 - Productos: 120
[Scroll 2] Scroll height: 31224 - Productos: 168
[Scroll 3] Scroll height: 38664 - Productos: 216
[Scroll 4] Scroll height: 46104 - Productos: 264
[Scroll 5] Scroll height: 53544 - Productos: 312
[Scroll 6] Scroll height: 60984 - Productos: 360
[Scroll 7] Scroll height: 68424 - Productos: 408
[Scroll 8] Scroll height: 75864 - Productos: 456
[Scroll 9] Scroll height: 83304 - Productos: 504
[Scroll 10] Scroll height: 90744 - Productos: 552
[Scroll 11] Scroll height: 94480 - Productos: 576
[Scroll 12] Scroll height: 101904 - Productos: 624
[Scroll 13] Scroll height: 109344 - Productos: 672
[Scroll 14] Scroll height: 113080 - Productos: 696
[Scroll 15] Scroll height: 120504 - Productos: 744
[Scroll 16] Scroll height: 120648 - Productos: 744
[Scroll 17] Scroll height: 124240 - Productos: 768
[Scroll 18] Scroll height: 127960 - Productos: 792
[Scroll 19] Scroll height: 

,categoria,subcategoria,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,fecha_extraccion,...,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,tipo_brocha,formulacion,tipo_piel,cobertura,acabado,textura
0,Ojos,Sombras de ojos,Rem Beauty,Starlet,Sombra de ojos líquida,23.00,1,2.0,12,2025-04-21,...,Clean at Sephora,"Mate, Irisado/Nacarado, Brillante",None,None,None,None,Todo tipo de pieles,None,None,Liquido
1,Labios,Voluminizador labios,Dior Backstage,Dior Lip Glow Oil,Aceite para labios brillante nutritivo realzad...,48.00,437,4.2,3,2025-04-21,...,None,None,None,None,None,None,Todo tipo de pieles,None,None,Aceite
2,Uñas,Esmalte de uñas,Dior,Dior Vernis,Laca de uñas efecto gel color couture,37.00,0,0.0,2,2025-04-21,...,None,None,None,None,None,None,Todo tipo de pieles,None,Purpurina,None
3,Labios,Gloss/ Brillo labial,Valentino Make Up,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...",39.99,6,4.7,5,2025-04-21,...,None,None,None,None,None,None,None,None,None,Liquido
4,Labios,Gloss/ Brillo labial,Fenty Beauty,Gloss Bomb Oil,Aceite-gloss para los labios,28.00,0,0.0,7,2025-04-21,...,None,None,None,None,None,None,Todo tipo de pieles,None,None,Liquido
